<a href="https://colab.research.google.com/github/shtengelM/FinalProjectAppliedMachineLearning2021/blob/main/Maayan_Guy_machine_learning_RLN_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Final Project Applied Machine Learning 2021**

#Students:

#Guy Zamostiano  204158976 
#Maayan Shtengel 316408988



# Evaluation of perforamce of the method introduced in the paper:

# “Regularization Learning Networks: Deep Learning for Tabular Datasets” 

# by: 	
# Ira Shavitt  Weizmann Institute of Science

#Eran Segal Weizman Institute of Science

# arXiv:1805.06440v3 [stat.ML] 23 Oct 2018
# https://arxiv.org/abs/1805.06440

#based on the papar's code: https://github.com/irashavitt/regularization_learning_networks


#Run Experiment on 20 datasets

In [ ]:
# download RLN model code 
import gdown
output = "Keras_implementation.py"
url = "https://drive.google.com/uc?id=1wPo1Qd0Q4XL2qHm_HqqWsvMtV-GMqnhc&export=download"
gdown.download(url, output, quiet=False)


# Authenticate and create the PyDrive client for saving experiments results on drive.
!pip install --upgrade gupload
!pip install pydrive --upgrade
from pydrive.auth import GoogleAuth
from google.colab import auth
import pandas as pd
auth.authenticate_user()

Downloading...
From: https://drive.google.com/uc?id=1wPo1Qd0Q4XL2qHm_HqqWsvMtV-GMqnhc&export=download
To: /content/Keras_implementation.py
100%|██████████| 2.94k/2.94k [00:00<00:00, 929kB/s]


Requirement already up-to-date: gupload in /usr/local/lib/python2.7/dist-packages (1.1.0)
Requirement already up-to-date: pydrive in /usr/local/lib/python2.7/dist-packages (1.3.1)


In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasRegressor
from keras import regularizers
from Keras_implementation import RLNCallback
from keras.models import Sequential
from keras.layers import Dense
from numpy.random import seed
from keras.backend import eval as keras_eval
from numpy.random import rand
from numpy.random import randint
from numpy.random import random_sample
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")
import time
import numpy as np
import itertools
from math import sqrt
from sklearn.metrics import mean_absolute_error, explained_variance_score , mean_squared_error, median_absolute_error



Using TensorFlow backend.


In [ ]:
# Final report for the experiments results
df_report = pd.DataFrame(columns = ['Dataset Name', 'Algorithm Name', 'Cross Validation [1-10]', 'Hyper-Parameters Values',
                                    'Mean Squared Error', 'Mean Absolute Error', 'Median Absolute Error', 'Max Error',
                                    'Explained Variance', 'Training time', 'Inference time for 1000 instances'])

In [ ]:
# Final report statistic test
df_report_statistical_test = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])

In [ ]:
def read_dataset(url, output):
  gdown.download(url, output, quiet=False)
  dataset = pd.read_csv(output)
  # print dataset
  return dataset

In [ ]:
def split_data(X_dataset, Y_dataset, random_state_param):
  X_train, X_test, y_train, y_test = train_test_split(X_dataset, Y_dataset, random_state=random_state_param)
  # print type(X_train)
  # print type(y_train)
  return X_train, X_test, y_train, y_test

In [ ]:
def max_error_imp(y_test, pred):
  max_dif = 0
  for y_expected, y_pred in zip(y_test,pred):
    dif = abs(y_expected-y_pred)
    if dif>max_dif:
      max_dif = dif
    
  return max_dif

In [ ]:
def measures(y_test, pred):
  mae = mean_absolute_error(y_test, pred)
  evs = explained_variance_score(y_test, pred)
  max_error = max_error_imp(y_test, pred)
  mse = mean_squared_error(y_test, pred)
  median_ae = median_absolute_error(y_test, pred)


  return mae, evs, max_error, mse, median_ae



## Create and run model
Create a basic Keras model and a test function

In [ ]:
def base_model(input_dim_model,layers=4, l1=0):
    assert layers > 1

    def build_fn():
        inner_l1 = l1
        # create model
        model = Sequential()
        # Construct the layers of the model to form a geometric series         
        prev_width = input_dim_model
        for width in np.exp(np.log(input_dim_model) * np.arange(layers - 1, 0, -1) / layers):
            width = int(np.round(width))
            model.add(Dense(width, input_dim=prev_width, kernel_initializer='glorot_normal', activation='relu', 
                            kernel_regularizer=regularizers.l1(inner_l1)))
            # For efficiency we only regularized the first layer            
            inner_l1 = 0
            prev_width = width
            
        model.add(Dense(1, kernel_initializer='glorot_normal'))
        
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='rmsprop')
        return model
    return build_fn


# Test model
# Method gets the model + train and test sets 
# Method create ensemble of the model -> runs num_repeates models and calculates the mean values for the measures 
# return the mean values of all the measures as the values for running the model
def test_model(X_train, y_train, X_test, y_test, build_fn, model_name, num_repeates=2):
    MJTCP = 32292 # Michael Jordan total career points
    seed(MJTCP)
    results = np.zeros(num_repeates)

    mae_arr = np.zeros(num_repeates)
    evs_arr = np.zeros(num_repeates)
    max_error_arr = np.zeros(num_repeates)
    mse_arr = np.zeros(num_repeates)
    median_ae_arr = np.zeros(num_repeates)

    accumulate_training = 0
    accumulate_testing = 0

    for i in range(num_repeates):
        reg = KerasRegressor(build_fn=build_fn, epochs=5, batch_size=10, verbose=0)

        start_training = time.time()
        reg.fit(X_train, y_train)
        end_training = time.time()

        training_time = (end_training - start_training)
        accumulate_training = accumulate_training + training_time
        
        start_testing = time.time()
        pred = reg.predict(X_test)
        end_testing = time.time()

        testing_time = (end_testing - start_testing)
        accumulate_testing = accumulate_testing + testing_time

        mae_arr[i], evs_arr[i], max_error_arr[i], mse_arr[i], median_ae_arr[i] = measures(y_test, pred)

    mae_mean, mae_std = mae_arr.mean(), mae_arr.std()
    evs_mean, evs_std = evs_arr.mean(), evs_arr.std()
    max_error_mean, max_error_std = max_error_arr.mean(), max_error_arr.std()
    mse_mean, mse_std = mse_arr.mean(), mse_arr.std()
    median_ae_mean, median_ae_std = median_ae_arr.mean(), median_ae_arr.std()

    accumulate_testing = (accumulate_testing*(1000.0))/(X_test.shape[0]*(1.0))

    return (mae_mean, mae_std), (evs_mean, evs_std), (max_error_mean, max_error_std), (mse_mean, mse_std), (median_ae_mean, median_ae_std), (accumulate_training, accumulate_testing)

## Unregularized network - find best depth

In [ ]:
def unregularized_network_best_depth(input_dim_model, X_train, y_train, X_test, y_test):
  print "start unregularized_network_best_depth"
  layers = 1

  prev_mse_mean = np.inf
  mse_mean = None

  while (mse_mean < prev_mse_mean) or (prev_mse_mean is None):
      prev_mse_mean = mse_mean
      layers += 1
      (mae_mean, mae_std), (evs_mean, evs_std), (max_error_mean, max_error_std), (mse_mean, mse_std), (median_ae_mean, median_ae_std), (accumulate_training, accumulate_testing_relative) = test_model(X_train, y_train, X_test, y_test, base_model(input_dim_model, layers=layers), "Network with %d layers" % layers)
      
  layers -= 1
  return layers

## L1 regularization of the network:

In [ ]:
# Ensemble L1 regularization network
# With hyper parameters optimization
# Run L1 regularization_network model on the train and test given for each hyper parameter
# Return the loss value of MSE for each hyper parameter
def L1_Regularization_network(hyper_parameters_l1, layers, input_dim_model, X_train, y_train, X_test, y_test):
  print "start L1 regularization"
  #l1 = 0.001

  mse_each_hyperparameter = np.zeros(len(hyper_parameters_l1))

  for i in range(len(hyper_parameters_l1)):

      #l1 *= 10

      #print "run L1_regularization_network f or l1 = %.0E" % hyper_parameters_l1[i]
      (mae_mean, mae_std), (evs_mean, evs_std), (max_error_mean, max_error_std), (mse_mean, mse_std), (median_ae_mean, median_ae_std), (accumulate_training, accumulate_testing_relative) = test_model(X_train, y_train, X_test, y_test, base_model(input_dim_model, layers=layers, l1=hyper_parameters_l1[i]), "L1 regularization of %.0E" % hyper_parameters_l1[i])
      #if mse_mean < prev_mse_mean:
      # (prev_mae_mean, prev_mae_std), (prev_evs_mean, prev_evs_std), (prev_max_error_mean, prev_max_error_std), (prev_mse_mean, prev_mse_std), (prev_median_ae_mean, prev_median_ae_std) = (mae_mean, mae_std), (evs_mean, evs_std), (max_error_mean, max_error_std), (mse_mean, mse_std), (median_ae_mean, median_ae_std)

      mse_each_hyperparameter[i] = mse_mean
      #cur_score = abs(cur_score)

  #best_l1_score = prev_mse_mean

  #l1 /= 10
  #print "The best L1 regularization is achieved with l1 = %.0E" % l1 

  return mse_each_hyperparameter #l1, (prev_mae_mean, prev_mae_std), (prev_evs_mean, prev_evs_std), (prev_max_error_mean, prev_max_error_std), (prev_mse_mean, prev_mse_std), (prev_median_ae_mean, prev_median_ae_std) 


## RLN

In [ ]:
# The Keras RLN implementation uses callbacks to change the weights of the layer. The callback gets the specific layer it regularizes as a parameter, and is passed to the fit function:

def RLN(input_dim_model, layers=4, **rln_kwargs):
    def build_fn():
        model = base_model(input_dim_model,layers=layers)()
        
        # For efficiency we only regularized the first layer
        rln_callback = RLNCallback(model.layers[0], **rln_kwargs)

        # Change the fit function of the model to except rln_callback:
        orig_fit = model.fit
        def rln_fit(*args, **fit_kwargs):
            orig_callbacks = fit_kwargs.get('callbacks', [])
            rln_callbacks = orig_callbacks + [rln_callback]
            return orig_fit(*args, callbacks=rln_callbacks, **fit_kwargs)

        model.fit = rln_fit

        return model

    return build_fn

In [ ]:
# Our improvement for thr RLN:
# The improvement we suggest is to use a technique of initializing weights of the deep neural network when creating the network in a way that will improve the performance of the model.
# We decided to use the “He initialization” method that is recommended for ‘ReLU’ by Jason Brownlee PhD in https://machinelearningmastery.com/weight-initialization-for-deep-learning-neural-networks/ .

def RLN_improved(input_dim_model, layers=4, **rln_kwargs):
    def build_fn():
        model = base_model(input_dim_model,layers=layers)()
        
        # calculate the range for the weights
        std = sqrt(2.0 / input_dim_model)
        # generate random numbers
        numbers = random_sample(model.layers[0].get_weights()[0].shape)
        # scale to the desired range
        scaled = numbers * std
        # all baises zeros as before
        bias = model.layers[0].get_weights()[1]
        model.layers[0].set_weights([scaled,bias])    



        # For efficiency we only regularized the first layer
        rln_callback = RLNCallback(model.layers[0], **rln_kwargs)

        # Change the fit function of the model to except rln_callback:
        orig_fit = model.fit
        def rln_fit(*args, **fit_kwargs):
            orig_callbacks = fit_kwargs.get('callbacks', [])
            rln_callbacks = orig_callbacks + [rln_callback]
            return orig_fit(*args, callbacks=rln_callbacks, **fit_kwargs)

        model.fit = rln_fit

        return model

    return build_fn

In [ ]:
# Apply RLN
# With hyper parameters optimization
# Run ensemble RLN network model on the train and test given for each hyper parameters combination
# Return the loss value of MSE for each hyper parameter combination
def apply_RLN(hyper_parameters, layers, input_dim_model, X_train, y_train, X_test, y_test, improvment):
  print "start apply RLN"

  mse_each_hyperparameter_combination = np.zeros(len(hyper_parameters))
  i = 0;
  # for cur_Theta, log_learning_rate in [(-8, 6), (-10, 5), (-10, 6), (-10, 7), (-12, 6)]:
  for cur_Theta, log_learning_rate in hyper_parameters:
          #random_params = {k: random.sample(v, 1)[0] for k, v in param_grid.items()}
      cur_learning_rate = np.power(10, log_learning_rate)

      if improvment:
        print("Apply RLN with improvment")
        (mae_mean, mae_std), (evs_mean, evs_std), (max_error_mean, max_error_std), (mse_mean, mse_std), (median_ae_mean, median_ae_std), (accumulate_training, accumulate_testing_relative) = test_model(X_train, y_train, X_test, y_test, RLN_improved(input_dim_model, layers=layers, norm=1, avg_reg=cur_Theta, learning_rate=cur_learning_rate), 
                              "RLN with Theta=%s and learning_rate=%.1E" % (cur_Theta, cur_learning_rate))
      else:
       # print("Apply RLN")
        (mae_mean, mae_std), (evs_mean, evs_std), (max_error_mean, max_error_std), (mse_mean, mse_std), (median_ae_mean, median_ae_std), (accumulate_training, accumulate_testing_relative) = test_model(X_train, y_train, X_test, y_test, RLN(input_dim_model, layers=layers, norm=1, avg_reg=cur_Theta, learning_rate=cur_learning_rate), 
                              "RLN with Theta=%s and learning_rate=%.1E" % (cur_Theta, cur_learning_rate))
      
      mse_each_hyperparameter_combination[i] = mse_mean
      i= i+1

  return mse_each_hyperparameter_combination #Theta, (mae_mean_of_best, mae_std_of_best), (evs_mean_of_best, evs_std_of_best), (max_error_mean_of_best, max_error_std_of_best), (mse_mean_of_best, mse_std_of_best), (median_ae_mean_of_best, median_ae_std_of_best)  

## L1: Evaluation by Cross Validation
  Internal (within Train) 3-fold Cross Validation – for hyperparameter 
  optimization
  
  External 10-fold Cross validation - to separate between the Train and Test 
  and reporting the performance metrics.

In [ ]:
def L1_Inner_CV(layers, INPUT_DIM, X,Y, hyper_parameters_l1):
  # configure the cross-validation procedure
  K_split = 3
  cv_folds = KFold(n_splits= K_split, shuffle=True, random_state=1)

  mse_each_hyperparameter_2D_array = np.zeros(shape=(K_split,len(hyper_parameters_l1)))  # for each split of data we have array of mse - the array length as number of hyper_parameters_l1

  i = 0;
  for train_ix, test_ix in cv_folds.split(X):
    print "#####################################L1 Inner CV number %d##############################################" %(i+1)
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = Y[train_ix], Y[test_ix]
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train, y_train)
    X_test = scaler.transform(X_test, y_test)

    #L1
    mse_each_hyperparameter_2D_array[i] = L1_Regularization_network(hyper_parameters_l1, layers, INPUT_DIM, X_train, y_train, X_test, y_test)
    print "##################################### END L1 Inner CV number %d##############################################" %(i+1)
    i=i+1

  #Calculate average for each hyper parameter through all splits
  MSE_each_hyperparameter_L1 = np.average(mse_each_hyperparameter_2D_array, axis=0);

  #best L1 will give the minimal MSE
  idx_best_l1 = np.argmin(MSE_each_hyperparameter_L1, axis=0)

  return hyper_parameters_l1[idx_best_l1]

In [ ]:
def L1_Outer_CV(layers, INPUT_DIM, X,Y, df_report, dataset_name):
  # configure the cross-validation procedure
  K_split = 5
  cv_outer = KFold(n_splits= K_split, shuffle=True, random_state=1)

  best_l1s = np.zeros(K_split)

  # measures for evaluation
  # columns 0,1 (mae_mean, mae_std)
  # columns 2,3 (evs_mean, evs_std)
  # columns 4,5 (max_error_mean, max_error_std)
  # columns 6,7 (mse_mean, mse_std)
  # columns 8,9 (median_ae_mean, median_ae_std)
  # columns 10,11 (accumulate_training, accumulate_testing_relative)
  measrues = np.zeros(shape=(K_split,12)) # rows- splits, columns- each measure


  # Define hyper parameters by Random Search with the help of implementation of Jason Brownlee PhD in https://machinelearningmastery.com/random-search-and-grid-search-for-function-optimization/
  # define range
  l1_min, l1_max = 0.001, 1

  # generate random values for l1
  num_hyperparameter_l1 = 4
  hyper_parameters_l1 = l1_min + rand(num_hyperparameter_l1) * (l1_max - l1_min)

  i = 0;
  for train_ix, test_ix in cv_outer.split(X):
    print "**********************************************L1 Outer CV number %d**************************************************************" %(i+1)
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = Y[train_ix], Y[test_ix]

    # Inner cross validation for hyperparameters optimization
    best_l1s[i] = L1_Inner_CV(layers, INPUT_DIM, X_train,y_train, hyper_parameters_l1)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train, y_train)
    X_test = scaler.transform(X_test, y_test)

    #trigger test_model with best hyperparameter for inner cross validation step
    (measrues[i][0], measrues[i][1]), (measrues[i][2], measrues[i][3]), (measrues[i][4], measrues[i][5]), (measrues[i][6], measrues[i][7]), (measrues[i][8], measrues[i][9]),(measrues[i][10], measrues[i][11]) = test_model(X_train, y_train, X_test, y_test, base_model(INPUT_DIM, layers=layers, l1=best_l1s[i]), "L1 regularization of %.0E" % best_l1s[i])
    record = {'Dataset Name': dataset_name, 'Algorithm Name': "L1", 'Cross Validation [1-10]': i+1, 'Hyper-Parameters Values': best_l1s[i],
                                    'Mean Squared Error': measrues[i][6], 'Mean Absolute Error' : measrues[i][0], 'Median Absolute Error' : measrues[i][8], 'Max Error' : measrues[i][4],
                                    'Explained Variance' : measrues[i][2], 'Training time' : measrues[i][10], 'Inference time for 1000 instances': measrues[i][11]}

    print "**********************************************END L1 Outer CV number %d**************************************************************" %(i+1)
    df_report = df_report.append(record, ignore_index = True)
    i=i+1

  mean_measures = np.average(measrues, axis=0);
  print mean_measures

  return df_report, mean_measures #index 6 contains Mean MSE

## RLN: Evaluation by Cross Validation
  Internal (within Train) 3-fold Cross Validation – for hyperparameter 
  optimization
  
  External 10-fold Cross validation - to separate between the Train and Test 
  and reporting the performance metrics.

In [ ]:
def RLN_Inner_CV(layers, INPUT_DIM, X,Y, hyper_parameters, improvment):
  # configure the cross-validation procedure
  K_split = 3
  cv_folds = KFold(n_splits= K_split, shuffle=True, random_state=1)

  mse_each_hyperparameter_combination_2D_array = np.zeros(shape=(K_split,len(hyper_parameters)))  # row - one split of data, columns: mse for each hyperparameters combination

  i = 0;
  for train_ix, test_ix in cv_folds.split(X):
    print "#####################################RLN Inner CV number %d##############################################" %(i+1)
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = Y[train_ix], Y[test_ix]

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train, y_train)
    X_test = scaler.transform(X_test, y_test)

    # RLN
    mse_each_hyperparameter_combination_2D_array[i] = apply_RLN(hyper_parameters, layers, INPUT_DIM, X_train, y_train, X_test, y_test, improvment)
    print "##################################### END RLN Inner CV number %d##############################################" %(i+1)
    i=i+1

  #Calculate average for each hyperparameters combination through all splits
  MSE_each_hyperparameters_combination = np.average(mse_each_hyperparameter_combination_2D_array, axis=0);

  #best hyperparameters combination that will give the minimal MSE
  idx_best_hyperparameters_combination = np.argmin(MSE_each_hyperparameters_combination, axis=0)

  return hyper_parameters[idx_best_hyperparameters_combination]

In [ ]:
def RLN_Outer_CV(layers, INPUT_DIM, X,Y, df_report, dataset_name, improvment):
  # configure the cross-validation procedure
  K_split = 5
  cv_outer = KFold(n_splits= K_split, shuffle=True, random_state=1)

  # measures for evaluation
  # columns 0,1 (mae_mean, mae_std)
  # columns 2,3 (evs_mean, evs_std)
  # columns 4,5 (max_error_mean, max_error_std)
  # columns 6,7 (mse_mean, mse_std)
  # columns 8,9 (median_ae_mean, median_ae_std)
  # columns 10,11 (accumulate_training, accumulate_testing_relative)
  measures = np.zeros(shape=(K_split,12)) # rows- splits, columns- each measure


 # Define hyper parameters by Random Search with the help of implementation of Jason Brownlee PhD in https://machinelearningmastery.com/random-search-and-grid-search-for-function-optimization/
  # Hyperparameters for RLN are: Theta and learning rate

  # From the "Keras_implementation.py" file of the Paper written: 
  #     typically have many weights in the network, and we optimize the coefficients in the log scale, optimal learning
  #     rates tend to be large, with best results between 10^4-10^6.
  
  # define range
  Theta_min, Theta_max = -12, -8

  # generate range of values for Theta and log_learning_Rate
  hyperparam_Theta = np.linspace(-12.0, -8.0, num=50) #Theta_min + rand(4) * (Theta_max - Theta_min)
  hyperparam_log_learning_rate = np.arange(4,7) #randint(low = log_learning_rate_min, high = log_learning_rate_max+1, size = 4)  #TODO: rand int only ??????????????????

  # Create all combinations of the 2 hyperparameters array of pairs [[Theta, log learning rate]]
  grid_hyperparameters = np.transpose([np.tile(hyperparam_Theta, len(hyperparam_log_learning_rate)), np.repeat(hyperparam_log_learning_rate, len(hyperparam_Theta))])

  #number of combinations to run in hyper parameters tuning
  num_combinations_to_run = 4
  #choose randomly from all combinations
  indexes = range(len(grid_hyperparameters))
  rand_idx = np.random.choice(indexes,num_combinations_to_run, replace = False)

  random_hyperparameters = grid_hyperparameters[rand_idx]

  best_hyperparam_combinations_each_outer_iter = np.zeros(shape=(K_split, 2)) # row: split of data, columns: [Theta, log learning rate]

  i = 0;
  for train_ix, test_ix in cv_outer.split(X):
    print "**********************************************RLN Outer CV number %d**************************************************************" %(i+1)
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = Y[train_ix], Y[test_ix]

    # Inner cross validation for hyperparameters optimization
    best_hyperparam_combinations_each_outer_iter[i] = RLN_Inner_CV(layers, INPUT_DIM, X_train,y_train, random_hyperparameters, improvment)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train, y_train)
    X_test = scaler.transform(X_test, y_test)

    # Trigger test_model with best hyperparameter for inner cross validation step
    cur_Theta = best_hyperparam_combinations_each_outer_iter[i][0]
    cur_learning_rate = best_hyperparam_combinations_each_outer_iter[i][1]

    alg_name = "RLN"
    if improvment:
      alg_name = " RLN improvement"
      (measures[i][0], measures[i][1]), (measures[i][2], measures[i][3]), (measures[i][4], measures[i][5]), (measures[i][6], measures[i][7]), (measures[i][8], measures[i][9]),(measures[i][10], measures[i][11]) = test_model(X_train, y_train, X_test, y_test, RLN_improved(INPUT_DIM, layers=layers, norm=1, avg_reg=cur_Theta, learning_rate=cur_learning_rate), 
                              "RLN Improvement with Theta=%s and learning_rate=%.1E" % (cur_Theta, cur_learning_rate))
    else:
      (measures[i][0], measures[i][1]), (measures[i][2], measures[i][3]), (measures[i][4], measures[i][5]), (measures[i][6], measures[i][7]), (measures[i][8], measures[i][9]),(measures[i][10], measures[i][11]) = test_model(X_train, y_train, X_test, y_test, RLN(INPUT_DIM, layers=layers, norm=1, avg_reg=cur_Theta, learning_rate=cur_learning_rate), 
                            "RLN with Theta=%s and learning_rate=%.1E" % (cur_Theta, cur_learning_rate))

    #Add to final report as record
    record = {'Dataset Name': dataset_name, 'Algorithm Name': alg_name, 'Cross Validation [1-10]': i+1, 'Hyper-Parameters Values': best_hyperparam_combinations_each_outer_iter[i],
                                    'Mean Squared Error': measures[i][6], 'Mean Absolute Error' : measures[i][0], 'Median Absolute Error' : measures[i][8], 'Max Error' : measures[i][4],
                                    'Explained Variance' : measures[i][2], 'Training time' : measures[i][10], 'Inference time for 1000 instances': measures[i][11]}

    df_report = df_report.append(record, ignore_index = True)
    print "**********************************************END RLN Outer CV number %d**************************************************************" %(i+1)
    i=i+1

  mean_measures = np.average(measures, axis=0);

  return df_report, mean_measures

Noise features

In [ ]:
# Add noisy features
noise_features = 10

# Data 1 - vineyard

In [ ]:
dataset_vineyard = read_dataset("https://drive.google.com/uc?id=1GMZ1hSEoshN9jBzzNXjs_gfale9NJENl&export=download", "vineyard.csv")
# https://drive.google.com/file/d/1GMZ1hSEoshN9jBzzNXjs_gfale9NJENl/view?usp=sharing
print(dataset_vineyard.isnull().values.any())

X_dataset_vineyard = dataset_vineyard.iloc[:,0:2].to_numpy()
y_dataset_vineyard = dataset_vineyard['lugs_1991'].to_numpy()

# Add noisy features
X_dataset_vineyard = np.concatenate([X_dataset_vineyard, np.random.normal(size=(X_dataset_vineyard.shape[0], noise_features))], axis=1)

INPUT_DIM_vineyard = X_dataset_vineyard.shape[1]

df_report, mean_measures_L1_vineyard = L1_Outer_CV(3, INPUT_DIM_vineyard, X_dataset_vineyard, y_dataset_vineyard, df_report, "vineyard")
df_report, mean_measures_RLN_vineyard = RLN_Outer_CV(3, INPUT_DIM_vineyard, X_dataset_vineyard, y_dataset_vineyard, df_report, "vineyard", False)
#With improvment
df_report, mean_measures_RLN_improvment_vineyard = RLN_Outer_CV(3, INPUT_DIM_vineyard, X_dataset_vineyard, y_dataset_vineyard, df_report, "vineyard", True)

#Update record mean mse for later statistical test
record_statitic_df_vineyard = {'Dataset Name': "vineyard", 'Mean Squared Error L1': mean_measures_L1_vineyard[6], 'Mean Squared Error RLN':mean_measures_RLN_vineyard[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_vineyard[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_vineyard, ignore_index = True)

df_report_statistical_test_vineyard = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_vineyard = df_report_statistical_test_vineyard.append(record_statitic_df_vineyard, ignore_index = True)
df_report.to_csv("df_report_vineyard.csv", sep=',')
df_report_statistical_test_vineyard.to_csv("df_report_statistical_test_vineyard.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_vineyard.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_vineyard.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1GMZ1hSEoshN9jBzzNXjs_gfale9NJENl&export=download
To: /content/vineyard.csv
100%|██████████| 524/524 [00:00<00:00, 224kB/s]


False
dataset_vineyard
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1*************************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,vineyard,L1,1,0.751829,307.187849,17.378607,17.077202,20.882986,0.031821,0.777454,3.552459
1,vineyard,L1,2,0.938725,323.641307,17.073572,19.945864,23.005481,0.005409,1.078673,3.891360
2,vineyard,L1,3,0.956744,355.251453,18.255944,19.586465,23.827490,-0.138691,0.777486,4.040718
3,vineyard,L1,4,0.751829,359.311897,18.659883,19.555291,23.114076,0.092595,0.795738,4.363298
4,vineyard,L1,5,0.751829,416.403123,19.872800,20.494006,26.510322,-0.052861,0.837453,4.817486
5,vineyard,RLN,1,"[-11.428571428571429, 4.0]",309.890194,17.446262,16.951485,20.960973,-0.032327,1.927192,4.681457
6,vineyard,RLN,2,"[-10.857142857142858, 4.0]",338.214194,17.522606,20.110615,23.919523,0.035231,1.877546,4.803918
7,vineyard,RLN,3,"[-8.16326530612245, 6.0]",350.329384,18.177494,19.207607,23.701715,-0.032584,2.011990,5.989385
8,vineyard,RLN,4,"[-8.16326530612245, 6.0]",314.036583,17.274358,18.165981,22.567699,-0.066022,1.909171,5.782127
9,vineyard,RLN,5,"[-11.428571428571429, 4.0]",415.011755,19.879853,20.945826,25.583652,0.026944,1.933852,6.169796


#Data 2 - triazines

In [ ]:
dataset_triazines = read_dataset("https://drive.google.com/uc?id=1OGbKlYLNWSi94ksJnOiorpRQgy1AflTH&export=download", "triazines.csv")
# https://drive.google.com/file/d/1OGbKlYLNWSi94ksJnOiorpRQgy1AflTH/view?usp=sharing

print(dataset_triazines.isnull().values.any())

X_dataset_triazines = dataset_triazines.iloc[:,0:58].to_numpy()
y_dataset_triazines = dataset_triazines['activity'].to_numpy()

# Add noisy features
X_dataset_triazines = np.concatenate([X_dataset_triazines, np.random.normal(size=(X_dataset_triazines.shape[0], noise_features))], axis=1)

INPUT_DIM_triazines = X_dataset_triazines.shape[1]

df_report, mean_measures_L1_triazines = L1_Outer_CV(3, INPUT_DIM_triazines, X_dataset_triazines, y_dataset_triazines, df_report, "triazines")
df_report, mean_measures_RLN_triazines = RLN_Outer_CV(3, INPUT_DIM_triazines, X_dataset_triazines, y_dataset_triazines, df_report, "triazines", False)
#With improvment
df_report, mean_measures_RLN_improvment_triazines = RLN_Outer_CV(3, INPUT_DIM_triazines, X_dataset_triazines, y_dataset_triazines, df_report, "triazines", True)

#Update record mean mse for later statistical test
record_statitic_df_triazines = {'Dataset Name': "triazines", 'Mean Squared Error L1': mean_measures_L1_triazines[6], 'Mean Squared Error RLN':mean_measures_RLN_triazines[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_triazines[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_triazines, ignore_index = True)

df_report_statistical_test_triazines = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_triazines = df_report_statistical_test_triazines.append(record_statitic_df_triazines, ignore_index = True)
df_report.to_csv("df_report_triazines.csv", sep=',')
df_report_statistical_test_triazines.to_csv("df_report_statistical_test_triazines.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_triazines.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_triazines.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1OGbKlYLNWSi94ksJnOiorpRQgy1AflTH&export=download
To: /content/triazines.csv
100%|██████████| 46.8k/46.8k [00:00<00:00, 26.1MB/s]


False
dataset_triazines
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1************************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,vineyard,L1,1,0.751829,3.071878e+02,1.737861e+01,17.077202,2.088299e+01,3.182129e-02,0.777454,3.552459
1,vineyard,L1,2,0.938725,3.236413e+02,1.707357e+01,19.945864,2.300548e+01,5.408541e-03,1.078673,3.891360
2,vineyard,L1,3,0.956744,3.552515e+02,1.825594e+01,19.586465,2.382749e+01,-1.386907e-01,0.777486,4.040718
3,vineyard,L1,4,0.751829,3.593119e+02,1.865988e+01,19.555291,2.311408e+01,9.259467e-02,0.795738,4.363298
4,vineyard,L1,5,0.751829,4.164031e+02,1.987280e+01,20.494006,2.651032e+01,-5.286134e-02,0.837453,4.817486
5,vineyard,RLN,1,"[-11.428571428571429, 4.0]",3.098902e+02,1.744626e+01,16.951485,2.096097e+01,-3.232749e-02,1.927192,4.681457
6,vineyard,RLN,2,"[-10.857142857142858, 4.0]",3.382142e+02,1.752261e+01,20.110615,2.391952e+01,3.523055e-02,1.877546,4.803918
7,vineyard,RLN,3,"[-8.16326530612245, 6.0]",3.503294e+02,1.817749e+01,19.207607,2.370171e+01,-3.258424e-02,2.011990,5.989385
8,vineyard,RLN,4,"[-8.16326530612245, 6.0]",3.140366e+02,1.727436e+01,18.165981,2.256770e+01,-6.602172e-02,1.909171,5.782127
9,vineyard,RLN,5,"[-11.428571428571429, 4.0]",4.150118e+02,1.987985e+01,20.945826,2.558365e+01,2.694366e-02,1.933852,6.169796


# Data 3 - aids

In [ ]:
dataset_aids = read_dataset("https://drive.google.com/uc?id=15HLgWMiKKr59q2gnrxcLb0uHZBs_1KsH&export=download", "analcatdata_gsssexsurvey.csv")
# https://drive.google.com/file/d/15HLgWMiKKr59q2gnrxcLb0uHZBs_1KsH/view?usp=sharing
print(dataset_aids.isnull().values.any())
dataset_aids = dataset_aids.dropna()

X_dataset_aids = dataset_aids.iloc[:,0:9].to_numpy()
y_dataset_aids = dataset_aids['AIDS_know'].to_numpy()

# Add noisy features
X_dataset_aids = np.concatenate([X_dataset_aids, np.random.normal(size=(X_dataset_aids.shape[0], noise_features))], axis=1)

INPUT_DIM_aids = X_dataset_aids.shape[1]

df_report, mean_measures_L1_aids = L1_Outer_CV(3, INPUT_DIM_aids, X_dataset_aids, y_dataset_aids, df_report, "aids")
df_report, mean_measures_RLN_aids = RLN_Outer_CV(3, INPUT_DIM_aids, X_dataset_aids, y_dataset_aids, df_report, "aids", False)
#With improvment
df_report, mean_measures_RLN_improvment_aids = RLN_Outer_CV(3, INPUT_DIM_aids, X_dataset_aids, y_dataset_aids, df_report, "aids", True)

#Update record mean mse for later statistical test
record_statitic_df_aids = {'Dataset Name': "aids", 'Mean Squared Error L1': mean_measures_L1_aids[6], 'Mean Squared Error RLN':mean_measures_RLN_aids[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_aids[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_aids, ignore_index = True)

df_report_statistical_test_aids = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_aids = df_report_statistical_test_aids.append(record_statitic_df_aids, ignore_index = True)
df_report.to_csv("df_report_aids.csv", sep=',')
df_report_statistical_test_aids.to_csv("df_report_statistical_test_aids.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_aids.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_aids.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=15HLgWMiKKr59q2gnrxcLb0uHZBs_1KsH&export=download
To: /content/analcatdata_gsssexsurvey.csv
100%|██████████| 4.33k/4.33k [00:00<00:00, 1.80MB/s]


True
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1**************************************************************
**********************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,aids,L1,1,0.20889,0.604524,0.436770,0.069760,1.930240,0.000000,1.093671,1.420490
1,aids,L1,2,0.221608,0.246669,0.289398,0.113037,1.886963,0.000000,1.155850,1.544341
2,aids,L1,3,0.968793,0.257975,0.257627,0.062204,1.937796,0.000000,1.176384,1.643687
3,aids,L1,4,0.221608,1.339761,0.513685,0.079884,3.920116,0.000000,1.181250,1.682162
4,aids,L1,5,0.968793,0.729249,0.421763,0.063215,2.936785,0.000000,1.199708,2.135323
5,aids,L1,1,0.818115,0.909780,0.660213,0.355655,2.273797,-0.937171,1.286726,2.447974
6,aids,L1,2,0.596042,1.382059,0.780840,0.559953,3.940537,-0.389132,1.364822,2.508386
7,aids,L1,3,0.380689,0.787668,0.511187,0.161108,3.233884,-0.133935,1.366028,2.563484
8,aids,L1,4,0.380689,0.675395,0.509394,0.369362,2.710130,-0.330623,1.350280,2.521062
9,aids,L1,5,0.380689,0.856214,0.460655,0.187313,4.189520,-0.260887,1.355303,2.611732


# Data 4 - analcatdata_negotiation

In [ ]:
dataset_analcatdata_negotiation = read_dataset("https://drive.google.com/uc?id=1LUrSpl2_tTdar0PUEStKqx8m3UY65p5C&export=download", "analcatdata_negotiation.csv")
# https://drive.google.com/file/d/1LUrSpl2_tTdar0PUEStKqx8m3UY65p5C/view?usp=sharing
print(dataset_analcatdata_negotiation.isnull().values.any())
dataset_analcatdata_negotiation = dataset_analcatdata_negotiation.dropna()


X_dataset_analcatdata_negotiation = dataset_analcatdata_negotiation.iloc[:,1:5].to_numpy()
y_dataset_analcatdata_negotiation = dataset_analcatdata_negotiation['Future_business'].to_numpy()

# Add noisy features
X_dataset_analcatdata_negotiation = np.concatenate([X_dataset_analcatdata_negotiation, np.random.normal(size=(X_dataset_analcatdata_negotiation.shape[0], noise_features))], axis=1)

INPUT_DIM_analcatdata_negotiation = X_dataset_analcatdata_negotiation.shape[1]

df_report, mean_measures_L1_analcatdata_negotiation = L1_Outer_CV(3, INPUT_DIM_analcatdata_negotiation, X_dataset_analcatdata_negotiation, y_dataset_analcatdata_negotiation, df_report, "analcatdata_negotiation")
df_report, mean_measures_RLN_analcatdata_negotiation = RLN_Outer_CV(3, INPUT_DIM_analcatdata_negotiation, X_dataset_analcatdata_negotiation, y_dataset_analcatdata_negotiation, df_report, "analcatdata_negotiation", False)
#With improvment
df_report, mean_measures_RLN_improvment_analcatdata_negotiation = RLN_Outer_CV(3, INPUT_DIM_analcatdata_negotiation, X_dataset_analcatdata_negotiation, y_dataset_analcatdata_negotiation, df_report, "analcatdata_negotiation", True)

#Update record mean mse for later statistical test
record_statitic_df_analcatdata_negotiation = {'Dataset Name': "analcatdata_negotiation", 'Mean Squared Error L1': mean_measures_L1_analcatdata_negotiation[6], 'Mean Squared Error RLN':mean_measures_RLN_analcatdata_negotiation[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_analcatdata_negotiation[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_analcatdata_negotiation, ignore_index = True)

df_report_statistical_test_analcatdata_negotiation = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_analcatdata_negotiation = df_report_statistical_test_analcatdata_negotiation.append(record_statitic_df_analcatdata_negotiation, ignore_index = True)
df_report.to_csv("df_report_analcatdata_negotiation.csv", sep=',')
df_report_statistical_test_analcatdata_negotiation.to_csv("df_report_statistical_test_analcatdata_negotiation.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_analcatdata_negotiation.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_analcatdata_negotiation.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1LUrSpl2_tTdar0PUEStKqx8m3UY65p5C&export=download
To: /content/analcatdata_negotiation.csv
100%|██████████| 5.27k/5.27k [00:00<00:00, 2.48MB/s]


True
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1**************************************************************
**********************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,aids,L1,1,0.20889,0.604524,0.436770,0.069760,1.930240,0.000000,1.093671,1.420490
1,aids,L1,2,0.221608,0.246669,0.289398,0.113037,1.886963,0.000000,1.155850,1.544341
2,aids,L1,3,0.968793,0.257975,0.257627,0.062204,1.937796,0.000000,1.176384,1.643687
3,aids,L1,4,0.221608,1.339761,0.513685,0.079884,3.920116,0.000000,1.181250,1.682162
4,aids,L1,5,0.968793,0.729249,0.421763,0.063215,2.936785,0.000000,1.199708,2.135323
5,aids,L1,1,0.818115,0.909780,0.660213,0.355655,2.273797,-0.937171,1.286726,2.447974
6,aids,L1,2,0.596042,1.382059,0.780840,0.559953,3.940537,-0.389132,1.364822,2.508386
7,aids,L1,3,0.380689,0.787668,0.511187,0.161108,3.233884,-0.133935,1.366028,2.563484
8,aids,L1,4,0.380689,0.675395,0.509394,0.369362,2.710130,-0.330623,1.350280,2.521062
9,aids,L1,5,0.380689,0.856214,0.460655,0.187313,4.189520,-0.260887,1.355303,2.611732


# Data 5 - analcatdata_runshoes

In [ ]:
dataset_analcatdata_runshoes = read_dataset("https://drive.google.com/uc?id=150otM59P06SejVAKu02DJlCmK8EUbbB1&export=download", "analcatdata_runshoes.csv")
# https://drive.google.com/file/d/150otM59P06SejVAKu02DJlCmK8EUbbB1/view?usp=sharing
print(dataset_analcatdata_runshoes.isnull().values.any())
dataset_analcatdata_runshoes = dataset_analcatdata_runshoes.dropna()

dataset_analcatdata_runshoes
X_dataset_analcatdata_runshoes = dataset_analcatdata_runshoes.iloc[:,1:9].to_numpy()
y_dataset_analcatdata_runshoes = dataset_analcatdata_runshoes['Shoes'].to_numpy()

# Add noisy features
X_dataset_analcatdata_runshoes = np.concatenate([X_dataset_analcatdata_runshoes, np.random.normal(size=(X_dataset_analcatdata_runshoes.shape[0], noise_features))], axis=1)

INPUT_DIM_analcatdata_runshoes = X_dataset_analcatdata_runshoes.shape[1]

df_report, mean_measures_L1_analcatdata_runshoes = L1_Outer_CV(3, INPUT_DIM_analcatdata_runshoes, X_dataset_analcatdata_runshoes, y_dataset_analcatdata_runshoes, df_report, "analcatdata_runshoes")
df_report, mean_measures_RLN_analcatdata_runshoes = RLN_Outer_CV(3, INPUT_DIM_analcatdata_runshoes, X_dataset_analcatdata_runshoes, y_dataset_analcatdata_runshoes, df_report, "analcatdata_runshoes", False)
#With improvment
df_report, mean_measures_RLN_improvment_analcatdata_runshoes = RLN_Outer_CV(3, INPUT_DIM_analcatdata_runshoes, X_dataset_analcatdata_runshoes, y_dataset_analcatdata_runshoes, df_report, "analcatdata_runshoes", True)

#Update record mean mse for later statistical test
record_statitic_df_analcatdata_runshoes = {'Dataset Name': "analcatdata_runshoes", 'Mean Squared Error L1': mean_measures_L1_analcatdata_runshoes[6], 'Mean Squared Error RLN':mean_measures_RLN_analcatdata_runshoes[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_analcatdata_runshoes[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_analcatdata_runshoes, ignore_index = True)

df_report_statistical_test_analcatdata_runshoes = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_analcatdata_runshoes = df_report_statistical_test_analcatdata_runshoes.append(record_statitic_df_analcatdata_runshoes, ignore_index = True)
df_report.to_csv("df_report_analcatdata_runshoes.csv", sep=',')
df_report_statistical_test_analcatdata_runshoes.to_csv("df_report_statistical_test_analcatdata_runshoes.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_analcatdata_runshoes.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_analcatdata_runshoes.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=150otM59P06SejVAKu02DJlCmK8EUbbB1&export=download
To: /content/analcatdata_runshoes.csv
100%|██████████| 2.31k/2.31k [00:00<00:00, 1.11MB/s]


True
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1**************************************************************
**********************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,aids,L1,1,0.20889,0.604524,0.436770,0.069760,1.930240,0.000000,1.093671,1.420490
1,aids,L1,2,0.221608,0.246669,0.289398,0.113037,1.886963,0.000000,1.155850,1.544341
2,aids,L1,3,0.968793,0.257975,0.257627,0.062204,1.937796,0.000000,1.176384,1.643687
3,aids,L1,4,0.221608,1.339761,0.513685,0.079884,3.920116,0.000000,1.181250,1.682162
4,aids,L1,5,0.968793,0.729249,0.421763,0.063215,2.936785,0.000000,1.199708,2.135323
5,aids,L1,1,0.818115,0.909780,0.660213,0.355655,2.273797,-0.937171,1.286726,2.447974
6,aids,L1,2,0.596042,1.382059,0.780840,0.559953,3.940537,-0.389132,1.364822,2.508386
7,aids,L1,3,0.380689,0.787668,0.511187,0.161108,3.233884,-0.133935,1.366028,2.563484
8,aids,L1,4,0.380689,0.675395,0.509394,0.369362,2.710130,-0.330623,1.350280,2.521062
9,aids,L1,5,0.380689,0.856214,0.460655,0.187313,4.189520,-0.260887,1.355303,2.611732


#Data 6 - sensory

In [ ]:
dataset_sensory = read_dataset("https://drive.google.com/uc?id=1rPtmnJQoPdz6VR3vh5dew_szTJ4cFJvw&export=download", "sensory.csv")
# https://drive.google.com/file/d/1rPtmnJQoPdz6VR3vh5dew_szTJ4cFJvw/view?usp=sharing

print(dataset_sensory.isnull().values.any())

X_dataset_sensory = dataset_sensory.iloc[:,0:11].to_numpy()
y_dataset_sensory = dataset_sensory['Score'].to_numpy()

# Add noisy features
X_dataset_sensory = np.concatenate([X_dataset_sensory, np.random.normal(size=(X_dataset_sensory.shape[0], noise_features))], axis=1)

INPUT_DIM_sensory = X_dataset_sensory.shape[1]

df_report, mean_measures_L1_sensory = L1_Outer_CV(3, INPUT_DIM_sensory, X_dataset_sensory, y_dataset_sensory, df_report, "sensory")
df_report, mean_measures_RLN_sensory = RLN_Outer_CV(3, INPUT_DIM_sensory, X_dataset_sensory, y_dataset_sensory, df_report, "sensory", False)
#With improvment
df_report, mean_measures_RLN_improvment_sensory = RLN_Outer_CV(3, INPUT_DIM_sensory, X_dataset_sensory, y_dataset_sensory, df_report, "sensory", True)

#Update record mean mse for later statistical test
record_statitic_df_sensory = {'Dataset Name': "sensory", 'Mean Squared Error L1': mean_measures_L1_sensory[6], 'Mean Squared Error RLN':mean_measures_RLN_sensory[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_sensory[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_sensory, ignore_index = True)

df_report_statistical_test_sensory = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_sensory = df_report_statistical_test_sensory.append(record_statitic_df_sensory, ignore_index = True)
df_report.to_csv("df_report_sensory.csv", sep=',')
df_report_statistical_test_sensory.to_csv("df_report_statistical_test_sensory.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_sensory.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_sensory.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1rPtmnJQoPdz6VR3vh5dew_szTJ4cFJvw&export=download
To: /content/sensory.csv
100%|██████████| 15.7k/15.7k [00:00<00:00, 8.69MB/s]


False
dataset_sensory
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1**************************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,vineyard,L1,1,0.751829,3.071878e+02,1.737861e+01,17.077202,2.088299e+01,3.182129e-02,0.777454,3.552459
1,vineyard,L1,2,0.938725,3.236413e+02,1.707357e+01,19.945864,2.300548e+01,5.408541e-03,1.078673,3.891360
2,vineyard,L1,3,0.956744,3.552515e+02,1.825594e+01,19.586465,2.382749e+01,-1.386907e-01,0.777486,4.040718
3,vineyard,L1,4,0.751829,3.593119e+02,1.865988e+01,19.555291,2.311408e+01,9.259467e-02,0.795738,4.363298
4,vineyard,L1,5,0.751829,4.164031e+02,1.987280e+01,20.494006,2.651032e+01,-5.286134e-02,0.837453,4.817486
5,vineyard,RLN,1,"[-11.428571428571429, 4.0]",3.098902e+02,1.744626e+01,16.951485,2.096097e+01,-3.232749e-02,1.927192,4.681457
6,vineyard,RLN,2,"[-10.857142857142858, 4.0]",3.382142e+02,1.752261e+01,20.110615,2.391952e+01,3.523055e-02,1.877546,4.803918
7,vineyard,RLN,3,"[-8.16326530612245, 6.0]",3.503294e+02,1.817749e+01,19.207607,2.370171e+01,-3.258424e-02,2.011990,5.989385
8,vineyard,RLN,4,"[-8.16326530612245, 6.0]",3.140366e+02,1.727436e+01,18.165981,2.256770e+01,-6.602172e-02,1.909171,5.782127
9,vineyard,RLN,5,"[-11.428571428571429, 4.0]",4.150118e+02,1.987985e+01,20.945826,2.558365e+01,2.694366e-02,1.933852,6.169796


#Data 7 - rmftsa_ladata

In [ ]:
dataset_rmftsa_ladata = read_dataset("https://drive.google.com/uc?id=1rn6oixrPjCzoSueVzLS016QEnjChjWCC&export=download", "rmftsa_ladata.csv")
# https://drive.google.com/file/d/1rn6oixrPjCzoSueVzLS016QEnjChjWCC/view?usp=sharing

print(dataset_rmftsa_ladata.isnull().values.any())

X_dataset_rmftsa_ladata = dataset_rmftsa_ladata.iloc[:,1:10].to_numpy()
y_dataset_rmftsa_ladata = dataset_rmftsa_ladata['Total_Mortality'].to_numpy()



# Add noisy features
X_dataset_rmftsa_ladata = np.concatenate([X_dataset_rmftsa_ladata, np.random.normal(size=(X_dataset_rmftsa_ladata.shape[0], noise_features))], axis=1)

INPUT_DIM_rmftsa_ladata = X_dataset_rmftsa_ladata.shape[1]

df_report, mean_measures_L1_rmftsa_ladata = L1_Outer_CV(3, INPUT_DIM_rmftsa_ladata, X_dataset_rmftsa_ladata, y_dataset_rmftsa_ladata, df_report, "rmftsa_ladata")
df_report, mean_measures_RLN_rmftsa_ladata = RLN_Outer_CV(3, INPUT_DIM_rmftsa_ladata, X_dataset_rmftsa_ladata, y_dataset_rmftsa_ladata, df_report, "rmftsa_ladata", False)
#With improvment
df_report, mean_measures_RLN_improvment_rmftsa_ladata = RLN_Outer_CV(3, INPUT_DIM_rmftsa_ladata, X_dataset_rmftsa_ladata, y_dataset_rmftsa_ladata, df_report, "rmftsa_ladata", True)

#Update record mean mse for later statistical test
record_statitic_df_rmftsa_ladata = {'Dataset Name': "rmftsa_ladata", 'Mean Squared Error L1': mean_measures_L1_rmftsa_ladata[6], 'Mean Squared Error RLN':mean_measures_RLN_rmftsa_ladata[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_rmftsa_ladata[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_rmftsa_ladata, ignore_index = True)

df_report_statistical_test_rmftsa_ladata = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_rmftsa_ladata = df_report_statistical_test_rmftsa_ladata.append(record_statitic_df_rmftsa_ladata, ignore_index = True)
df_report.to_csv("df_report_rmftsa_ladata.csv", sep=',')
df_report_statistical_test_rmftsa_ladata.to_csv("df_report_statistical_test_rmftsa_ladata.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_rmftsa_ladata.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_rmftsa_ladata.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1rn6oixrPjCzoSueVzLS016QEnjChjWCC&export=download
To: /content/rmftsa_ladata.csv
100%|██████████| 32.3k/32.3k [00:00<00:00, 15.3MB/s]


False
dataset_rmftsa_ladata
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1********************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,vineyard,L1,1,0.751829,3.071878e+02,1.737861e+01,17.077202,2.088299e+01,3.182129e-02,0.777454,3.552459
1,vineyard,L1,2,0.938725,3.236413e+02,1.707357e+01,19.945864,2.300548e+01,5.408541e-03,1.078673,3.891360
2,vineyard,L1,3,0.956744,3.552515e+02,1.825594e+01,19.586465,2.382749e+01,-1.386907e-01,0.777486,4.040718
3,vineyard,L1,4,0.751829,3.593119e+02,1.865988e+01,19.555291,2.311408e+01,9.259467e-02,0.795738,4.363298
4,vineyard,L1,5,0.751829,4.164031e+02,1.987280e+01,20.494006,2.651032e+01,-5.286134e-02,0.837453,4.817486
5,vineyard,RLN,1,"[-11.428571428571429, 4.0]",3.098902e+02,1.744626e+01,16.951485,2.096097e+01,-3.232749e-02,1.927192,4.681457
6,vineyard,RLN,2,"[-10.857142857142858, 4.0]",3.382142e+02,1.752261e+01,20.110615,2.391952e+01,3.523055e-02,1.877546,4.803918
7,vineyard,RLN,3,"[-8.16326530612245, 6.0]",3.503294e+02,1.817749e+01,19.207607,2.370171e+01,-3.258424e-02,2.011990,5.989385
8,vineyard,RLN,4,"[-8.16326530612245, 6.0]",3.140366e+02,1.727436e+01,18.165981,2.256770e+01,-6.602172e-02,1.909171,5.782127
9,vineyard,RLN,5,"[-11.428571428571429, 4.0]",4.150118e+02,1.987985e+01,20.945826,2.558365e+01,2.694366e-02,1.933852,6.169796


#Data 8 - pwLinear

In [ ]:
dataset_pwLinear = read_dataset("https://drive.google.com/uc?id=1z7oGcZiSFQ22xlX9iGAv7QqfYEKrrmgS&export=download", "pwLinear.csv")
# https://drive.google.com/file/d/1z7oGcZiSFQ22xlX9iGAv7QqfYEKrrmgS/view?usp=sharing
print(dataset_pwLinear.isnull().values.any())

X_dataset_pwLinear = dataset_pwLinear.iloc[:,0:10].to_numpy()
y_dataset_pwLinear = dataset_pwLinear['class'].to_numpy()

# Add noisy features
X_dataset_pwLinear = np.concatenate([X_dataset_pwLinear, np.random.normal(size=(X_dataset_pwLinear.shape[0], noise_features))], axis=1)

INPUT_DIM_pwLinear = X_dataset_pwLinear.shape[1]

df_report, mean_measures_L1_pwLinear = L1_Outer_CV(3, INPUT_DIM_pwLinear, X_dataset_pwLinear, y_dataset_pwLinear, df_report, "pwLinear")
df_report, mean_measures_RLN_pwLinear = RLN_Outer_CV(3, INPUT_DIM_pwLinear, X_dataset_pwLinear, y_dataset_pwLinear, df_report, "pwLinear", False)
#With improvment
df_report, mean_measures_RLN_improvment_pwLinear = RLN_Outer_CV(3, INPUT_DIM_pwLinear, X_dataset_pwLinear, y_dataset_pwLinear, df_report, "pwLinear", True)

#Update record mean mse for later statistical test
record_statitic_df_pwLinear = {'Dataset Name': "pwLinear", 'Mean Squared Error L1': mean_measures_L1_pwLinear[6], 'Mean Squared Error RLN':mean_measures_RLN_pwLinear[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_pwLinear[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_pwLinear, ignore_index = True)

df_report_statistical_test_pwLinear = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_pwLinear = df_report_statistical_test_pwLinear.append(record_statitic_df_pwLinear, ignore_index = True)
df_report.to_csv("df_report_pwLinear.csv", sep=',')
df_report_statistical_test_pwLinear.to_csv("df_report_statistical_test_pwLinear.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_pwLinear.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_pwLinear.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1z7oGcZiSFQ22xlX9iGAv7QqfYEKrrmgS&export=download
To: /content/pwLinear.csv
100%|██████████| 5.86k/5.86k [00:00<00:00, 2.09MB/s]


False
dataset_pwLinear
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1*************************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,vineyard,L1,1,0.751829,3.071878e+02,1.737861e+01,17.077202,2.088299e+01,3.182129e-02,0.777454,3.552459
1,vineyard,L1,2,0.938725,3.236413e+02,1.707357e+01,19.945864,2.300548e+01,5.408541e-03,1.078673,3.891360
2,vineyard,L1,3,0.956744,3.552515e+02,1.825594e+01,19.586465,2.382749e+01,-1.386907e-01,0.777486,4.040718
3,vineyard,L1,4,0.751829,3.593119e+02,1.865988e+01,19.555291,2.311408e+01,9.259467e-02,0.795738,4.363298
4,vineyard,L1,5,0.751829,4.164031e+02,1.987280e+01,20.494006,2.651032e+01,-5.286134e-02,0.837453,4.817486
5,vineyard,RLN,1,"[-11.428571428571429, 4.0]",3.098902e+02,1.744626e+01,16.951485,2.096097e+01,-3.232749e-02,1.927192,4.681457
6,vineyard,RLN,2,"[-10.857142857142858, 4.0]",3.382142e+02,1.752261e+01,20.110615,2.391952e+01,3.523055e-02,1.877546,4.803918
7,vineyard,RLN,3,"[-8.16326530612245, 6.0]",3.503294e+02,1.817749e+01,19.207607,2.370171e+01,-3.258424e-02,2.011990,5.989385
8,vineyard,RLN,4,"[-8.16326530612245, 6.0]",3.140366e+02,1.727436e+01,18.165981,2.256770e+01,-6.602172e-02,1.909171,5.782127
9,vineyard,RLN,5,"[-11.428571428571429, 4.0]",4.150118e+02,1.987985e+01,20.945826,2.558365e+01,2.694366e-02,1.933852,6.169796


#Data 9 - pharynx

In [ ]:
dataset_pharynx = read_dataset("https://drive.google.com/uc?id=11PANOTysPrNxUvX8awl3-LYvVwM12vve&export=download", "pharynx.csv")
# https://drive.google.com/file/d/11PANOTysPrNxUvX8awl3-LYvVwM12vve/view?usp=sharing
print(dataset_pharynx.isnull().values.any())
dataset_pharynx = dataset_pharynx.dropna()
print(dataset_pharynx.isnull().values.any())

X_dataset_pharynx = dataset_pharynx.iloc[:,0:11].to_numpy()
y_dataset_pharynx = dataset_pharynx['class'].to_numpy()

# Add noisy features
X_dataset_pharynx = np.concatenate([X_dataset_pharynx, np.random.normal(size=(X_dataset_pharynx.shape[0], noise_features))], axis=1)

INPUT_DIM_pharynx = X_dataset_pharynx.shape[1]

df_report, mean_measures_L1_pharynx = L1_Outer_CV(3, INPUT_DIM_pharynx, X_dataset_pharynx, y_dataset_pharynx, df_report, "pharynx")
df_report, mean_measures_RLN_pharynx = RLN_Outer_CV(3, INPUT_DIM_pharynx, X_dataset_pharynx, y_dataset_pharynx, df_report, "pharynx", False)
#With improvment
df_report, mean_measures_RLN_improvment_pharynx = RLN_Outer_CV(3, INPUT_DIM_pharynx, X_dataset_pharynx, y_dataset_pharynx, df_report, "pharynx", True)

#Update record mean mse for later statistical test
record_statitic_df_pharynx = {'Dataset Name': "pharynx", 'Mean Squared Error L1': mean_measures_L1_pharynx[6], 'Mean Squared Error RLN':mean_measures_RLN_pharynx[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_pharynx[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_pharynx, ignore_index = True)

df_report_statistical_test_pharynx = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_pharynx = df_report_statistical_test_pharynx.append(record_statitic_df_pharynx, ignore_index = True)
df_report.to_csv("df_report_pharynx.csv", sep=',')
df_report_statistical_test_pharynx.to_csv("df_report_statistical_test_pharynx.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_pharynx.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_pharynx.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=11PANOTysPrNxUvX8awl3-LYvVwM12vve&export=download
To: /content/pharynx.csv
100%|██████████| 6.27k/6.27k [00:00<00:00, 2.46MB/s]


True
False
dataset_pharynx
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1*********************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,vineyard,L1,1,0.751829,3.071878e+02,1.737861e+01,17.077202,2.088299e+01,3.182129e-02,0.777454,3.552459
1,vineyard,L1,2,0.938725,3.236413e+02,1.707357e+01,19.945864,2.300548e+01,5.408541e-03,1.078673,3.891360
2,vineyard,L1,3,0.956744,3.552515e+02,1.825594e+01,19.586465,2.382749e+01,-1.386907e-01,0.777486,4.040718
3,vineyard,L1,4,0.751829,3.593119e+02,1.865988e+01,19.555291,2.311408e+01,9.259467e-02,0.795738,4.363298
4,vineyard,L1,5,0.751829,4.164031e+02,1.987280e+01,20.494006,2.651032e+01,-5.286134e-02,0.837453,4.817486
5,vineyard,RLN,1,"[-11.428571428571429, 4.0]",3.098902e+02,1.744626e+01,16.951485,2.096097e+01,-3.232749e-02,1.927192,4.681457
6,vineyard,RLN,2,"[-10.857142857142858, 4.0]",3.382142e+02,1.752261e+01,20.110615,2.391952e+01,3.523055e-02,1.877546,4.803918
7,vineyard,RLN,3,"[-8.16326530612245, 6.0]",3.503294e+02,1.817749e+01,19.207607,2.370171e+01,-3.258424e-02,2.011990,5.989385
8,vineyard,RLN,4,"[-8.16326530612245, 6.0]",3.140366e+02,1.727436e+01,18.165981,2.256770e+01,-6.602172e-02,1.909171,5.782127
9,vineyard,RLN,5,"[-11.428571428571429, 4.0]",4.150118e+02,1.987985e+01,20.945826,2.558365e+01,2.694366e-02,1.933852,6.169796


#Data 10 - machine_cpu

In [ ]:
dataset_machine_cpu = read_dataset("https://drive.google.com/uc?id=1y0_KVu7PDOdtzajuHgTjImx_iir4uE2i&export=download", "machine_cpu.csv")
# https://drive.google.com/file/d/1y0_KVu7PDOdtzajuHgTjImx_iir4uE2i/view?usp=sharing
print(dataset_machine_cpu.isnull().values.any())

X_dataset_machine_cpu = dataset_machine_cpu.iloc[:,0:6].to_numpy()
y_dataset_machine_cpu = dataset_machine_cpu['class'].to_numpy()

# Add noisy features
X_dataset_machine_cpu = np.concatenate([X_dataset_machine_cpu, np.random.normal(size=(X_dataset_machine_cpu.shape[0], noise_features))], axis=1)

INPUT_DIM_machine_cpu = X_dataset_machine_cpu.shape[1]

df_report, mean_measures_L1_machine_cpu = L1_Outer_CV(3, INPUT_DIM_machine_cpu, X_dataset_machine_cpu, y_dataset_machine_cpu, df_report, "machine_cpu")
df_report, mean_measures_RLN_machine_cpu = RLN_Outer_CV(3, INPUT_DIM_machine_cpu, X_dataset_machine_cpu, y_dataset_machine_cpu, df_report, "machine_cpu", False)
#With improvment
df_report, mean_measures_RLN_improvment_machine_cpu = RLN_Outer_CV(3, INPUT_DIM_machine_cpu, X_dataset_machine_cpu, y_dataset_machine_cpu, df_report, "machine_cpu", True)

#Update record mean mse for later statistical test
record_statitic_df_machine_cpu = {'Dataset Name': "machine_cpu", 'Mean Squared Error L1': mean_measures_L1_machine_cpu[6], 'Mean Squared Error RLN':mean_measures_RLN_machine_cpu[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_machine_cpu[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_machine_cpu, ignore_index = True)

df_report_statistical_test_machine_cpu = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_machine_cpu = df_report_statistical_test_machine_cpu.append(record_statitic_df_machine_cpu, ignore_index = True)
df_report.to_csv("df_report_machine_cpu.csv", sep=',')
df_report_statistical_test_machine_cpu.to_csv("df_report_statistical_test_machine_cpu.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_machine_cpu.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_machine_cpu.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1y0_KVu7PDOdtzajuHgTjImx_iir4uE2i&export=download
To: /content/machine_cpu.csv
100%|██████████| 5.09k/5.09k [00:00<00:00, 2.45MB/s]


False
dataset_machine_cpu
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1**********************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,vineyard,L1,1,0.751829,3.071878e+02,1.737861e+01,17.077202,2.088299e+01,3.182129e-02,0.777454,3.552459
1,vineyard,L1,2,0.938725,3.236413e+02,1.707357e+01,19.945864,2.300548e+01,5.408541e-03,1.078673,3.891360
2,vineyard,L1,3,0.956744,3.552515e+02,1.825594e+01,19.586465,2.382749e+01,-1.386907e-01,0.777486,4.040718
3,vineyard,L1,4,0.751829,3.593119e+02,1.865988e+01,19.555291,2.311408e+01,9.259467e-02,0.795738,4.363298
4,vineyard,L1,5,0.751829,4.164031e+02,1.987280e+01,20.494006,2.651032e+01,-5.286134e-02,0.837453,4.817486
5,vineyard,RLN,1,"[-11.428571428571429, 4.0]",3.098902e+02,1.744626e+01,16.951485,2.096097e+01,-3.232749e-02,1.927192,4.681457
6,vineyard,RLN,2,"[-10.857142857142858, 4.0]",3.382142e+02,1.752261e+01,20.110615,2.391952e+01,3.523055e-02,1.877546,4.803918
7,vineyard,RLN,3,"[-8.16326530612245, 6.0]",3.503294e+02,1.817749e+01,19.207607,2.370171e+01,-3.258424e-02,2.011990,5.989385
8,vineyard,RLN,4,"[-8.16326530612245, 6.0]",3.140366e+02,1.727436e+01,18.165981,2.256770e+01,-6.602172e-02,1.909171,5.782127
9,vineyard,RLN,5,"[-11.428571428571429, 4.0]",4.150118e+02,1.987985e+01,20.945826,2.558365e+01,2.694366e-02,1.933852,6.169796


#Data 11 - liver-disorders

In [ ]:
dataset_liver_disorders = read_dataset("https://drive.google.com/uc?id=1fF_x3DsOspPFs2CT80istM3y4T9uyErX&export=download", "liver-disorders.csv")
# https://drive.google.com/file/d/1fF_x3DsOspPFs2CT80istM3y4T9uyErX/view?usp=sharing
print(dataset_liver_disorders.isnull().values.any())

X_dataset_liver_disorders = dataset_liver_disorders.iloc[:,0:5].to_numpy()
y_dataset_liver_disorders = dataset_liver_disorders['drinks'].to_numpy()

# Add noisy features
X_dataset_liver_disorders = np.concatenate([X_dataset_liver_disorders, np.random.normal(size=(X_dataset_liver_disorders.shape[0], noise_features))], axis=1)

INPUT_DIM_liver_disorders = X_dataset_liver_disorders.shape[1]

df_report, mean_measures_L1_liver_disorders = L1_Outer_CV(3, INPUT_DIM_liver_disorders, X_dataset_liver_disorders, y_dataset_liver_disorders, df_report, "liver_disorders")
df_report, mean_measures_RLN_liver_disorders = RLN_Outer_CV(3, INPUT_DIM_liver_disorders, X_dataset_liver_disorders, y_dataset_liver_disorders, df_report, "liver_disorders", False)
#With improvment
df_report, mean_measures_RLN_improvment_liver_disorders = RLN_Outer_CV(3, INPUT_DIM_liver_disorders, X_dataset_liver_disorders, y_dataset_liver_disorders, df_report, "liver_disorders", True)

#Update record mean mse for later statistical test
record_statitic_df_liver_disorders = {'Dataset Name': "liver_disorders", 'Mean Squared Error L1': mean_measures_L1_liver_disorders[6], 'Mean Squared Error RLN':mean_measures_RLN_liver_disorders[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_liver_disorders[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_liver_disorders, ignore_index = True)

df_report_statistical_test_liver_disorders = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_liver_disorders = df_report_statistical_test_liver_disorders.append(record_statitic_df_liver_disorders, ignore_index = True)
df_report.to_csv("df_report_liver_disorders.csv", sep=',')
df_report_statistical_test_liver_disorders.to_csv("df_report_statistical_test_liver_disorders.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_liver_disorders.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_liver_disorders.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1fF_x3DsOspPFs2CT80istM3y4T9uyErX&export=download
To: /content/liver-disorders.csv
100%|██████████| 6.52k/6.52k [00:00<00:00, 3.30MB/s]


False
dataset_liver_disorders
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1******************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,vineyard,L1,1,0.751829,3.071878e+02,1.737861e+01,17.077202,2.088299e+01,3.182129e-02,0.777454,3.552459
1,vineyard,L1,2,0.938725,3.236413e+02,1.707357e+01,19.945864,2.300548e+01,5.408541e-03,1.078673,3.891360
2,vineyard,L1,3,0.956744,3.552515e+02,1.825594e+01,19.586465,2.382749e+01,-1.386907e-01,0.777486,4.040718
3,vineyard,L1,4,0.751829,3.593119e+02,1.865988e+01,19.555291,2.311408e+01,9.259467e-02,0.795738,4.363298
4,vineyard,L1,5,0.751829,4.164031e+02,1.987280e+01,20.494006,2.651032e+01,-5.286134e-02,0.837453,4.817486
5,vineyard,RLN,1,"[-11.428571428571429, 4.0]",3.098902e+02,1.744626e+01,16.951485,2.096097e+01,-3.232749e-02,1.927192,4.681457
6,vineyard,RLN,2,"[-10.857142857142858, 4.0]",3.382142e+02,1.752261e+01,20.110615,2.391952e+01,3.523055e-02,1.877546,4.803918
7,vineyard,RLN,3,"[-8.16326530612245, 6.0]",3.503294e+02,1.817749e+01,19.207607,2.370171e+01,-3.258424e-02,2.011990,5.989385
8,vineyard,RLN,4,"[-8.16326530612245, 6.0]",3.140366e+02,1.727436e+01,18.165981,2.256770e+01,-6.602172e-02,1.909171,5.782127
9,vineyard,RLN,5,"[-11.428571428571429, 4.0]",4.150118e+02,1.987985e+01,20.945826,2.558365e+01,2.694366e-02,1.933852,6.169796


#Data 12 - kc1-numeric

In [ ]:
dataset_kc1_numeric = read_dataset("https://drive.google.com/uc?id=1RnpT7ebcUDOUiIGYMJOV69TMAseUlKFo&export=download", "kc1-numeric.csv")
# https://drive.google.com/file/d/1RnpT7ebcUDOUiIGYMJOV69TMAseUlKFo/view?usp=sharing
print(dataset_kc1_numeric.isnull().values.any())

X_dataset_kc1_numeric = dataset_kc1_numeric.iloc[:,0:86].to_numpy()
y_dataset_kc1_numeric = dataset_kc1_numeric['NUMDEFECTS'].to_numpy()

# Add noisy features
X_dataset_kc1_numeric = np.concatenate([X_dataset_kc1_numeric, np.random.normal(size=(X_dataset_kc1_numeric.shape[0], noise_features))], axis=1)

INPUT_DIM_kc1_numeric = X_dataset_kc1_numeric.shape[1]

df_report, mean_measures_L1_kc1_numeric = L1_Outer_CV(3, INPUT_DIM_kc1_numeric, X_dataset_kc1_numeric, y_dataset_kc1_numeric, df_report, "kc1_numeric")
df_report, mean_measures_RLN_kc1_numeric = RLN_Outer_CV(3, INPUT_DIM_kc1_numeric, X_dataset_kc1_numeric, y_dataset_kc1_numeric, df_report, "kc1_numeric", False)
#With improvment
df_report, mean_measures_RLN_improvment_kc1_numeric = RLN_Outer_CV(3, INPUT_DIM_kc1_numeric, X_dataset_kc1_numeric, y_dataset_kc1_numeric, df_report, "kc1_numeric", True)

#Update record mean mse for later statistical test
record_statitic_df_kc1_numeric = {'Dataset Name': "kc1_numeric", 'Mean Squared Error L1': mean_measures_L1_kc1_numeric[6], 'Mean Squared Error RLN':mean_measures_RLN_kc1_numeric[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_kc1_numeric[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_kc1_numeric, ignore_index = True)

df_report_statistical_test_kc1_numeric = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_kc1_numeric = df_report_statistical_test_kc1_numeric.append(record_statitic_df_kc1_numeric, ignore_index = True)
df_report.to_csv("df_report_kc1_numeric.csv", sep=',')
df_report_statistical_test_kc1_numeric.to_csv("df_report_statistical_test_kc1_numeric.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_kc1_numeric.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_kc1_numeric.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1RnpT7ebcUDOUiIGYMJOV69TMAseUlKFo&export=download
To: /content/kc1-numeric.csv
100%|██████████| 54.0k/54.0k [00:00<00:00, 8.12MB/s]


False
dataset_kc1_numeric
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1**********************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,kc1_numeric,L1,1,0.59182,56.353232,3.577932,0.832973,30.784375,-0.019732,0.884208,1.348413
1,kc1_numeric,L1,2,0.0939923,40.379562,3.512180,0.874388,19.646297,-0.377182,1.231910,1.463101
2,kc1_numeric,L1,3,0.0939923,96.412440,4.428279,0.714828,37.597858,0.095391,0.985722,1.671553
3,kc1_numeric,L1,4,0.0939923,217.459075,5.521369,0.896466,74.462950,0.454506,0.961105,1.665280
4,kc1_numeric,L1,5,0.619111,53.973454,4.017438,0.413447,21.641087,0.243652,0.988991,1.762037
5,kc1_numeric,RLN,1,"[-9.306122448979592, 6.0]",53.821837,3.257851,0.587679,31.110112,0.051428,11.733151,1.935030
6,kc1_numeric,RLN,2,"[-9.63265306122449, 6.0]",37.462463,3.313215,0.300616,19.938206,0.002976,11.784944,2.044965
7,kc1_numeric,RLN,3,"[-10.122448979591837, 6.0]",90.348424,4.229661,0.681615,36.589796,0.143673,12.465107,2.182484
8,kc1_numeric,RLN,4,"[-9.63265306122449, 6.0]",290.376603,6.180665,0.919864,84.994876,0.275196,12.178495,2.195424
9,kc1_numeric,RLN,5,"[-9.63265306122449, 6.0]",68.366722,4.473863,0.488034,23.804154,0.074276,12.318184,2.400892


#Data 13 - happinessRank_2015

In [ ]:
dataset_happinessRank_2015 = read_dataset("https://drive.google.com/uc?id=1E9lc1_Fr-_ynQV9YW8Fk_5pCa4A91rYE&export=download", "happinessRank_2015.csv")
print(dataset_happinessRank_2015.isnull().values.any())

# https://drive.google.com/file/d/1E9lc1_Fr-_ynQV9YW8Fk_5pCa4A91rYE/view?usp=sharing


X_dataset_happinessRank_2015 = dataset_happinessRank_2015.iloc[:,2:10].to_numpy()
y_dataset_happinessRank_2015 = dataset_happinessRank_2015['Happiness_Score'].to_numpy()

# Add noisy features
X_dataset_happinessRank_2015 = np.concatenate([X_dataset_happinessRank_2015, np.random.normal(size=(X_dataset_happinessRank_2015.shape[0], noise_features))], axis=1)

INPUT_DIM_happinessRank_2015 = X_dataset_happinessRank_2015.shape[1]

df_report, mean_measures_L1_happinessRank_2015 = L1_Outer_CV(3, INPUT_DIM_happinessRank_2015, X_dataset_happinessRank_2015, y_dataset_happinessRank_2015, df_report, "happinessRank_2015")
df_report, mean_measures_RLN_happinessRank_2015 = RLN_Outer_CV(3, INPUT_DIM_happinessRank_2015, X_dataset_happinessRank_2015, y_dataset_happinessRank_2015, df_report, "happinessRank_2015", False)
#With improvment
df_report, mean_measures_RLN_improvment_happinessRank_2015 = RLN_Outer_CV(3, INPUT_DIM_happinessRank_2015, X_dataset_happinessRank_2015, y_dataset_happinessRank_2015, df_report, "happinessRank_2015", True)

#Update record mean mse for later statistical test
record_statitic_df_happinessRank_2015 = {'Dataset Name': "happinessRank_2015", 'Mean Squared Error L1': mean_measures_L1_happinessRank_2015[6], 'Mean Squared Error RLN':mean_measures_RLN_happinessRank_2015[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_happinessRank_2015[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_happinessRank_2015, ignore_index = True)

df_report_statistical_test_happinessRank_2015 = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_happinessRank_2015 = df_report_statistical_test_happinessRank_2015.append(record_statitic_df_happinessRank_2015, ignore_index = True)
df_report.to_csv("df_report_happinessRank_2015.csv", sep=',')
df_report_statistical_test_happinessRank_2015.to_csv("df_report_statistical_test_happinessRank_2015.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_happinessRank_2015.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_happinessRank_2015.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1E9lc1_Fr-_ynQV9YW8Fk_5pCa4A91rYE&export=download
To: /content/happinessRank_2015.csv
100%|██████████| 15.6k/15.6k [00:00<00:00, 10.2MB/s]


False
dataset_happinessRank_2015
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1***************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,kc1_numeric,L1,1,0.59182,56.353232,3.577932,0.832973,30.784375,-0.019732,0.884208,1.348413
1,kc1_numeric,L1,2,0.0939923,40.379562,3.512180,0.874388,19.646297,-0.377182,1.231910,1.463101
2,kc1_numeric,L1,3,0.0939923,96.412440,4.428279,0.714828,37.597858,0.095391,0.985722,1.671553
3,kc1_numeric,L1,4,0.0939923,217.459075,5.521369,0.896466,74.462950,0.454506,0.961105,1.665280
4,kc1_numeric,L1,5,0.619111,53.973454,4.017438,0.413447,21.641087,0.243652,0.988991,1.762037
5,kc1_numeric,RLN,1,"[-9.306122448979592, 6.0]",53.821837,3.257851,0.587679,31.110112,0.051428,11.733151,1.935030
6,kc1_numeric,RLN,2,"[-9.63265306122449, 6.0]",37.462463,3.313215,0.300616,19.938206,0.002976,11.784944,2.044965
7,kc1_numeric,RLN,3,"[-10.122448979591837, 6.0]",90.348424,4.229661,0.681615,36.589796,0.143673,12.465107,2.182484
8,kc1_numeric,RLN,4,"[-9.63265306122449, 6.0]",290.376603,6.180665,0.919864,84.994876,0.275196,12.178495,2.195424
9,kc1_numeric,RLN,5,"[-9.63265306122449, 6.0]",68.366722,4.473863,0.488034,23.804154,0.074276,12.318184,2.400892


#Data 14 - echoMonths

In [ ]:
dataset_echoMonths = read_dataset("https://drive.google.com/uc?id=1PtV6MVGi1pBrl-L4moqZMGCoQwowkWVQ&export=download", "echoMonths.csv")
# https://drive.google.com/file/d/1PtV6MVGi1pBrl-L4moqZMGCoQwowkWVQ/view?usp=sharing
print(dataset_echoMonths.isnull().values.any())
dataset_echoMonths = dataset_echoMonths.dropna()
print(dataset_echoMonths.isnull().values.any())

X_dataset_echoMonths = dataset_echoMonths.iloc[:,0:9].to_numpy()
y_dataset_echoMonths = dataset_echoMonths['class'].to_numpy()

# Add noisy features
X_dataset_echoMonths = np.concatenate([X_dataset_echoMonths, np.random.normal(size=(X_dataset_echoMonths.shape[0], noise_features))], axis=1)

INPUT_DIM_echoMonths = X_dataset_echoMonths.shape[1]

df_report, mean_measures_L1_echoMonths = L1_Outer_CV(3, INPUT_DIM_echoMonths, X_dataset_echoMonths, y_dataset_echoMonths, df_report, "echoMonths")
df_report, mean_measures_RLN_echoMonths = RLN_Outer_CV(3, INPUT_DIM_echoMonths, X_dataset_echoMonths, y_dataset_echoMonths, df_report, "echoMonths", False)
#With improvment
df_report, mean_measures_RLN_improvment_echoMonths = RLN_Outer_CV(3, INPUT_DIM_echoMonths, X_dataset_echoMonths, y_dataset_echoMonths, df_report, "echoMonths", True)

#Update record mean mse for later statistical test
record_statitic_df_echoMonths = {'Dataset Name': "echoMonths", 'Mean Squared Error L1': mean_measures_L1_echoMonths[6], 'Mean Squared Error RLN':mean_measures_RLN_echoMonths[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_echoMonths[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_echoMonths, ignore_index = True)

df_report_statistical_test_echoMonths = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_echoMonths = df_report_statistical_test_echoMonths.append(record_statitic_df_echoMonths, ignore_index = True)
df_report.to_csv("df_report_echoMonths.csv", sep=',')
df_report_statistical_test_echoMonths.to_csv("df_report_statistical_test_echoMonths.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_echoMonths.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_echoMonths.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1PtV6MVGi1pBrl-L4moqZMGCoQwowkWVQ&export=download
To: /content/echoMonths.csv
100%|██████████| 4.34k/4.34k [00:00<00:00, 2.17MB/s]


True
False
dataset_echoMonths
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1******************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,kc1_numeric,L1,1,0.59182,56.353232,3.577932,0.832973,30.784375,-0.019732,0.884208,1.348413
1,kc1_numeric,L1,2,0.0939923,40.379562,3.512180,0.874388,19.646297,-0.377182,1.231910,1.463101
2,kc1_numeric,L1,3,0.0939923,96.412440,4.428279,0.714828,37.597858,0.095391,0.985722,1.671553
3,kc1_numeric,L1,4,0.0939923,217.459075,5.521369,0.896466,74.462950,0.454506,0.961105,1.665280
4,kc1_numeric,L1,5,0.619111,53.973454,4.017438,0.413447,21.641087,0.243652,0.988991,1.762037
5,kc1_numeric,RLN,1,"[-9.306122448979592, 6.0]",53.821837,3.257851,0.587679,31.110112,0.051428,11.733151,1.935030
6,kc1_numeric,RLN,2,"[-9.63265306122449, 6.0]",37.462463,3.313215,0.300616,19.938206,0.002976,11.784944,2.044965
7,kc1_numeric,RLN,3,"[-10.122448979591837, 6.0]",90.348424,4.229661,0.681615,36.589796,0.143673,12.465107,2.182484
8,kc1_numeric,RLN,4,"[-9.63265306122449, 6.0]",290.376603,6.180665,0.919864,84.994876,0.275196,12.178495,2.195424
9,kc1_numeric,RLN,5,"[-9.63265306122449, 6.0]",68.366722,4.473863,0.488034,23.804154,0.074276,12.318184,2.400892


#Data 15 - cholesterol

In [ ]:
dataset_cholesterol = read_dataset("https://drive.google.com/uc?id=1AUreLAJaITGVlEzdJNC9PC5S8lCh5Wvg&export=download", "cholesterol.csv")

# https://drive.google.com/file/d/1AUreLAJaITGVlEzdJNC9PC5S8lCh5Wvg/view?usp=sharing
print(dataset_cholesterol.isnull().values.any())
dataset_cholesterol = dataset_cholesterol.dropna()
print(dataset_cholesterol.isnull().values.any())

X_dataset_cholesterol = dataset_cholesterol.iloc[:,0:13].to_numpy()
y_dataset_cholesterol = dataset_cholesterol['chol'].to_numpy()

# Add noisy features
X_dataset_cholesterol = np.concatenate([X_dataset_cholesterol, np.random.normal(size=(X_dataset_cholesterol.shape[0], noise_features))], axis=1)

INPUT_DIM_cholesterol = X_dataset_cholesterol.shape[1]

df_report, mean_measures_L1_cholesterol = L1_Outer_CV(3, INPUT_DIM_cholesterol, X_dataset_cholesterol, y_dataset_cholesterol, df_report, "cholesterol")
df_report, mean_measures_RLN_cholesterol = RLN_Outer_CV(3, INPUT_DIM_cholesterol, X_dataset_cholesterol, y_dataset_cholesterol, df_report, "cholesterol", False)
#With improvment
df_report, mean_measures_RLN_improvment_cholesterol = RLN_Outer_CV(3, INPUT_DIM_cholesterol, X_dataset_cholesterol, y_dataset_cholesterol, df_report, "cholesterol", True)

#Update record mean mse for later statistical test
record_statitic_df_cholesterol = {'Dataset Name': "cholesterol", 'Mean Squared Error L1': mean_measures_L1_cholesterol[6], 'Mean Squared Error RLN':mean_measures_RLN_cholesterol[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_cholesterol[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_cholesterol, ignore_index = True)

df_report_statistical_test_cholesterol = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_cholesterol = df_report_statistical_test_cholesterol.append(record_statitic_df_cholesterol, ignore_index = True)
df_report.to_csv("df_report_cholesterol.csv", sep=',')
df_report_statistical_test_cholesterol.to_csv("df_report_statistical_test_cholesterol.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_cholesterol.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_cholesterol.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1AUreLAJaITGVlEzdJNC9PC5S8lCh5Wvg&export=download
To: /content/cholesterol.csv
100%|██████████| 11.3k/11.3k [00:00<00:00, 4.36MB/s]


True
False
dataset_cholesterol
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1*****************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,kc1_numeric,L1,1,0.59182,56.353232,3.577932,0.832973,30.784375,-0.019732,0.884208,1.348413
1,kc1_numeric,L1,2,0.0939923,40.379562,3.512180,0.874388,19.646297,-0.377182,1.231910,1.463101
2,kc1_numeric,L1,3,0.0939923,96.412440,4.428279,0.714828,37.597858,0.095391,0.985722,1.671553
3,kc1_numeric,L1,4,0.0939923,217.459075,5.521369,0.896466,74.462950,0.454506,0.961105,1.665280
4,kc1_numeric,L1,5,0.619111,53.973454,4.017438,0.413447,21.641087,0.243652,0.988991,1.762037
5,kc1_numeric,RLN,1,"[-9.306122448979592, 6.0]",53.821837,3.257851,0.587679,31.110112,0.051428,11.733151,1.935030
6,kc1_numeric,RLN,2,"[-9.63265306122449, 6.0]",37.462463,3.313215,0.300616,19.938206,0.002976,11.784944,2.044965
7,kc1_numeric,RLN,3,"[-10.122448979591837, 6.0]",90.348424,4.229661,0.681615,36.589796,0.143673,12.465107,2.182484
8,kc1_numeric,RLN,4,"[-9.63265306122449, 6.0]",290.376603,6.180665,0.919864,84.994876,0.275196,12.178495,2.195424
9,kc1_numeric,RLN,5,"[-9.63265306122449, 6.0]",68.366722,4.473863,0.488034,23.804154,0.074276,12.318184,2.400892


#Data 16 - baskball

In [ ]:
dataset_baskball = read_dataset("https://drive.google.com/uc?id=1QUHq_Czzvf4ACUhfA6-fL124knaqO5LP&export=download", "baskball.csv")

# https://drive.google.com/file/d/1QUHq_Czzvf4ACUhfA6-fL124knaqO5LP/view?usp=sharing
print(dataset_baskball.isnull().values.any())

X_dataset_baskball = dataset_baskball.iloc[:,0:4].to_numpy()
y_dataset_baskball = dataset_baskball['points_per_minute'].to_numpy()

# Add noisy features
X_dataset_baskball = np.concatenate([X_dataset_baskball, np.random.normal(size=(X_dataset_baskball.shape[0], noise_features))], axis=1)

INPUT_DIM_baskball = X_dataset_baskball.shape[1]

df_report, mean_measures_L1_baskball = L1_Outer_CV(3, INPUT_DIM_baskball, X_dataset_baskball, y_dataset_baskball, df_report, "baskball")
df_report, mean_measures_RLN_baskball = RLN_Outer_CV(3, INPUT_DIM_baskball, X_dataset_baskball, y_dataset_baskball, df_report, "baskball", False)
#With improvment
df_report, mean_measures_RLN_improvment_baskball = RLN_Outer_CV(3, INPUT_DIM_baskball, X_dataset_baskball, y_dataset_baskball, df_report, "baskball", True)

#Update record mean mse for later statistical test
record_statitic_df_baskball = {'Dataset Name': "baskball", 'Mean Squared Error L1': mean_measures_L1_baskball[6], 'Mean Squared Error RLN':mean_measures_RLN_baskball[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_baskball[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_baskball, ignore_index = True)

df_report_statistical_test_baskball = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_baskball = df_report_statistical_test_baskball.append(record_statitic_df_baskball, ignore_index = True)
df_report.to_csv("df_report_baskball.csv", sep=',')
df_report_statistical_test_baskball.to_csv("df_report_statistical_test_baskball.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_baskball.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_baskball.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1QUHq_Czzvf4ACUhfA6-fL124knaqO5LP&export=download
To: /content/baskball.csv
100%|██████████| 2.71k/2.71k [00:00<00:00, 2.04MB/s]


False
dataset_baskball
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1*************************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,kc1_numeric,L1,1,0.59182,56.353232,3.577932,0.832973,30.784375,-0.019732,0.884208,1.348413
1,kc1_numeric,L1,2,0.0939923,40.379562,3.512180,0.874388,19.646297,-0.377182,1.231910,1.463101
2,kc1_numeric,L1,3,0.0939923,96.412440,4.428279,0.714828,37.597858,0.095391,0.985722,1.671553
3,kc1_numeric,L1,4,0.0939923,217.459075,5.521369,0.896466,74.462950,0.454506,0.961105,1.665280
4,kc1_numeric,L1,5,0.619111,53.973454,4.017438,0.413447,21.641087,0.243652,0.988991,1.762037
5,kc1_numeric,RLN,1,"[-9.306122448979592, 6.0]",53.821837,3.257851,0.587679,31.110112,0.051428,11.733151,1.935030
6,kc1_numeric,RLN,2,"[-9.63265306122449, 6.0]",37.462463,3.313215,0.300616,19.938206,0.002976,11.784944,2.044965
7,kc1_numeric,RLN,3,"[-10.122448979591837, 6.0]",90.348424,4.229661,0.681615,36.589796,0.143673,12.465107,2.182484
8,kc1_numeric,RLN,4,"[-9.63265306122449, 6.0]",290.376603,6.180665,0.919864,84.994876,0.275196,12.178495,2.195424
9,kc1_numeric,RLN,5,"[-9.63265306122449, 6.0]",68.366722,4.473863,0.488034,23.804154,0.074276,12.318184,2.400892


#Data 17 - autoMpg

In [ ]:
dataset_autoMpg = read_dataset("https://drive.google.com/uc?id=1tWtn84L19_sSG5vgrGd4HMlOXDosAoBJ&export=download", "autoMpg.csv")
print(dataset_autoMpg.isnull().values.any())
dataset_autoMpg = dataset_autoMpg.dropna()
print(dataset_autoMpg.isnull().values.any())
# https://drive.google.com/file/d/1tWtn84L19_sSG5vgrGd4HMlOXDosAoBJ/view?usp=sharing

X_dataset_autoMpg = dataset_autoMpg.iloc[:,0:7].to_numpy()
y_dataset_autoMpg = dataset_autoMpg['class'].to_numpy()

# Add noisy features
X_dataset_autoMpg = np.concatenate([X_dataset_autoMpg, np.random.normal(size=(X_dataset_autoMpg.shape[0], noise_features))], axis=1)

INPUT_DIM_autoMpg = X_dataset_autoMpg.shape[1]

df_report, mean_measures_L1_autoMpg = L1_Outer_CV(3, INPUT_DIM_autoMpg, X_dataset_autoMpg, y_dataset_autoMpg, df_report, "autoMpg")
df_report, mean_measures_RLN_autoMpg = RLN_Outer_CV(3, INPUT_DIM_autoMpg, X_dataset_autoMpg, y_dataset_autoMpg, df_report, "autoMpg", False)
#With improvment
df_report, mean_measures_RLN_improvment_autoMpg = RLN_Outer_CV(3, INPUT_DIM_autoMpg, X_dataset_autoMpg, y_dataset_autoMpg, df_report, "autoMpg", True)

#Update record mean mse for later statistical test
record_statitic_df_autoMpg = {'Dataset Name': "autoMpg", 'Mean Squared Error L1': mean_measures_L1_autoMpg[6], 'Mean Squared Error RLN':mean_measures_RLN_autoMpg[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_autoMpg[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_autoMpg, ignore_index = True)

df_report_statistical_test_autoMpg = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_autoMpg = df_report_statistical_test_autoMpg.append(record_statitic_df_autoMpg, ignore_index = True)
df_report.to_csv("df_report_autoMpg.csv", sep=',')
df_report_statistical_test_autoMpg.to_csv("df_report_statistical_test_autoMpg.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_autoMpg.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_autoMpg.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1tWtn84L19_sSG5vgrGd4HMlOXDosAoBJ&export=download
To: /content/autoMpg.csv
100%|██████████| 11.3k/11.3k [00:00<00:00, 4.25MB/s]


True
False
dataset_autoMpg
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1*********************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,kc1_numeric,L1,1,0.59182,56.353232,3.577932,0.832973,30.784375,-0.019732,0.884208,1.348413
1,kc1_numeric,L1,2,0.0939923,40.379562,3.512180,0.874388,19.646297,-0.377182,1.231910,1.463101
2,kc1_numeric,L1,3,0.0939923,96.412440,4.428279,0.714828,37.597858,0.095391,0.985722,1.671553
3,kc1_numeric,L1,4,0.0939923,217.459075,5.521369,0.896466,74.462950,0.454506,0.961105,1.665280
4,kc1_numeric,L1,5,0.619111,53.973454,4.017438,0.413447,21.641087,0.243652,0.988991,1.762037
5,kc1_numeric,RLN,1,"[-9.306122448979592, 6.0]",53.821837,3.257851,0.587679,31.110112,0.051428,11.733151,1.935030
6,kc1_numeric,RLN,2,"[-9.63265306122449, 6.0]",37.462463,3.313215,0.300616,19.938206,0.002976,11.784944,2.044965
7,kc1_numeric,RLN,3,"[-10.122448979591837, 6.0]",90.348424,4.229661,0.681615,36.589796,0.143673,12.465107,2.182484
8,kc1_numeric,RLN,4,"[-9.63265306122449, 6.0]",290.376603,6.180665,0.919864,84.994876,0.275196,12.178495,2.195424
9,kc1_numeric,RLN,5,"[-9.63265306122449, 6.0]",68.366722,4.473863,0.488034,23.804154,0.074276,12.318184,2.400892


#Data 18 - auto_price

In [ ]:
dataset_auto_price = read_dataset("https://drive.google.com/uc?id=12Sj8aPkV_zfK6rlX1MW33ptpjwB515_9&export=download", "auto_price.csv")

# https://drive.google.com/file/d/12Sj8aPkV_zfK6rlX1MW33ptpjwB515_9/view?usp=sharing
print(dataset_auto_price.isnull().values.any())

X_dataset_auto_price = dataset_auto_price.iloc[:,0:15].to_numpy()
y_dataset_auto_price = dataset_auto_price['price'].to_numpy()

# Add noisy features
X_dataset_auto_price = np.concatenate([X_dataset_auto_price, np.random.normal(size=(X_dataset_auto_price.shape[0], noise_features))], axis=1)

INPUT_DIM_auto_price = X_dataset_auto_price.shape[1]

df_report, mean_measures_L1_auto_price = L1_Outer_CV(3, INPUT_DIM_auto_price, X_dataset_auto_price, y_dataset_auto_price, df_report, "auto_price")
df_report, mean_measures_RLN_auto_price = RLN_Outer_CV(3, INPUT_DIM_auto_price, X_dataset_auto_price, y_dataset_auto_price, df_report, "auto_price", False)
#With improvment
df_report, mean_measures_RLN_improvment_auto_price = RLN_Outer_CV(3, INPUT_DIM_auto_price, X_dataset_auto_price, y_dataset_auto_price, df_report, "auto_price", True)

#Update record mean mse for later statistical test
record_statitic_df_auto_price = {'Dataset Name': "auto_price", 'Mean Squared Error L1': mean_measures_L1_auto_price[6], 'Mean Squared Error RLN':mean_measures_RLN_auto_price[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_auto_price[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_auto_price, ignore_index = True)

df_report_statistical_test_auto_price = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_auto_price = df_report_statistical_test_auto_price.append(record_statitic_df_auto_price, ignore_index = True)
df_report.to_csv("df_report_auto_price.csv", sep=',')
df_report_statistical_test_auto_price.to_csv("df_report_statistical_test_auto_price.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_auto_price.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_auto_price.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=12Sj8aPkV_zfK6rlX1MW33ptpjwB515_9&export=download
To: /content/auto_price.csv
100%|██████████| 12.1k/12.1k [00:00<00:00, 22.2MB/s]


False
dataset_auto_price
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1***********************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,kc1_numeric,L1,1,0.59182,5.635323e+01,3.577932,0.832973,30.784375,-0.019732,0.884208,1.348413
1,kc1_numeric,L1,2,0.0939923,4.037956e+01,3.512180,0.874388,19.646297,-0.377182,1.231910,1.463101
2,kc1_numeric,L1,3,0.0939923,9.641244e+01,4.428279,0.714828,37.597858,0.095391,0.985722,1.671553
3,kc1_numeric,L1,4,0.0939923,2.174591e+02,5.521369,0.896466,74.462950,0.454506,0.961105,1.665280
4,kc1_numeric,L1,5,0.619111,5.397345e+01,4.017438,0.413447,21.641087,0.243652,0.988991,1.762037
5,kc1_numeric,RLN,1,"[-9.306122448979592, 6.0]",5.382184e+01,3.257851,0.587679,31.110112,0.051428,11.733151,1.935030
6,kc1_numeric,RLN,2,"[-9.63265306122449, 6.0]",3.746246e+01,3.313215,0.300616,19.938206,0.002976,11.784944,2.044965
7,kc1_numeric,RLN,3,"[-10.122448979591837, 6.0]",9.034842e+01,4.229661,0.681615,36.589796,0.143673,12.465107,2.182484
8,kc1_numeric,RLN,4,"[-9.63265306122449, 6.0]",2.903766e+02,6.180665,0.919864,84.994876,0.275196,12.178495,2.195424
9,kc1_numeric,RLN,5,"[-9.63265306122449, 6.0]",6.836672e+01,4.473863,0.488034,23.804154,0.074276,12.318184,2.400892


#Data 19 - analcatdata_gviolence

In [ ]:
dataset_analcatdata_gviolence = read_dataset("https://drive.google.com/uc?id=1_CKMkEC8gA7Lk1rlVysx-pRiaDPJ2x0G&export=download", "analcatdata_gviolence.csv")

# https://drive.google.com/file/d/1_CKMkEC8gA7Lk1rlVysx-pRiaDPJ2x0G/view?usp=sharing
print(dataset_analcatdata_gviolence.isnull().values.any())

X_dataset_analcatdata_gviolence = dataset_analcatdata_gviolence.iloc[:,0:8].to_numpy()
y_dataset_analcatdata_gviolence = dataset_analcatdata_gviolence['Violence_time'].to_numpy()

# Add noisy features
X_dataset_analcatdata_gviolence = np.concatenate([X_dataset_analcatdata_gviolence, np.random.normal(size=(X_dataset_analcatdata_gviolence.shape[0], noise_features))], axis=1)

INPUT_DIM_analcatdata_gviolence = X_dataset_analcatdata_gviolence.shape[1]

df_report, mean_measures_L1_analcatdata_gviolence = L1_Outer_CV(3, INPUT_DIM_analcatdata_gviolence, X_dataset_analcatdata_gviolence, y_dataset_analcatdata_gviolence, df_report, "analcatdata_gviolence")
df_report, mean_measures_RLN_analcatdata_gviolence = RLN_Outer_CV(3, INPUT_DIM_analcatdata_gviolence, X_dataset_analcatdata_gviolence, y_dataset_analcatdata_gviolence, df_report, "analcatdata_gviolence", False)
#With improvment
df_report, mean_measures_RLN_improvment_analcatdata_gviolence = RLN_Outer_CV(3, INPUT_DIM_analcatdata_gviolence, X_dataset_analcatdata_gviolence, y_dataset_analcatdata_gviolence, df_report, "analcatdata_gviolence", True)

#Update record mean mse for later statistical test
record_statitic_df_analcatdata_gviolence = {'Dataset Name': "analcatdata_gviolence", 'Mean Squared Error L1': mean_measures_L1_analcatdata_gviolence[6], 'Mean Squared Error RLN':mean_measures_RLN_analcatdata_gviolence[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_analcatdata_gviolence[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_analcatdata_gviolence, ignore_index = True)

df_report_statistical_test_analcatdata_gviolence = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_analcatdata_gviolence = df_report_statistical_test_analcatdata_gviolence.append(record_statitic_df_analcatdata_gviolence, ignore_index = True)
df_report.to_csv("df_report_analcatdata_gviolence.csv", sep=',')
df_report_statistical_test_analcatdata_gviolence.to_csv("df_report_statistical_test_analcatdata_gviolence.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_analcatdata_gviolence.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_analcatdata_gviolence.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1_CKMkEC8gA7Lk1rlVysx-pRiaDPJ2x0G&export=download
To: /content/analcatdata_gviolence.csv
100%|██████████| 2.12k/2.12k [00:00<00:00, 1.03MB/s]


False
dataset_analcatdata_gviolence
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1************************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,kc1_numeric,L1,1,0.59182,5.635323e+01,3.577932,0.832973,30.784375,-0.019732,0.884208,1.348413
1,kc1_numeric,L1,2,0.0939923,4.037956e+01,3.512180,0.874388,19.646297,-0.377182,1.231910,1.463101
2,kc1_numeric,L1,3,0.0939923,9.641244e+01,4.428279,0.714828,37.597858,0.095391,0.985722,1.671553
3,kc1_numeric,L1,4,0.0939923,2.174591e+02,5.521369,0.896466,74.462950,0.454506,0.961105,1.665280
4,kc1_numeric,L1,5,0.619111,5.397345e+01,4.017438,0.413447,21.641087,0.243652,0.988991,1.762037
5,kc1_numeric,RLN,1,"[-9.306122448979592, 6.0]",5.382184e+01,3.257851,0.587679,31.110112,0.051428,11.733151,1.935030
6,kc1_numeric,RLN,2,"[-9.63265306122449, 6.0]",3.746246e+01,3.313215,0.300616,19.938206,0.002976,11.784944,2.044965
7,kc1_numeric,RLN,3,"[-10.122448979591837, 6.0]",9.034842e+01,4.229661,0.681615,36.589796,0.143673,12.465107,2.182484
8,kc1_numeric,RLN,4,"[-9.63265306122449, 6.0]",2.903766e+02,6.180665,0.919864,84.994876,0.275196,12.178495,2.195424
9,kc1_numeric,RLN,5,"[-9.63265306122449, 6.0]",6.836672e+01,4.473863,0.488034,23.804154,0.074276,12.318184,2.400892


#Data 20 - Admission_Predict_kaggle

In [ ]:
dataset_Admission_Predict_kaggle = read_dataset("https://drive.google.com/uc?id=1Y8XB-m_7cMf5OOCqG41PA1OK9tQk10Rh&export=download", "Admission_Predict_kaggle.csv")

# https://drive.google.com/file/d/1Y8XB-m_7cMf5OOCqG41PA1OK9tQk10Rh/view?usp=sharing
print(dataset_Admission_Predict_kaggle.isnull().values.any())

X_dataset_Admission_Predict_kaggle = dataset_Admission_Predict_kaggle.iloc[:,0:7].to_numpy()
y_dataset_Admission_Predict_kaggle = dataset_Admission_Predict_kaggle['Chance_of_Admit '].to_numpy()

# Add noisy features
X_dataset_Admission_Predict_kaggle = np.concatenate([X_dataset_Admission_Predict_kaggle, np.random.normal(size=(X_dataset_Admission_Predict_kaggle.shape[0], noise_features))], axis=1)

INPUT_DIM_Admission_Predict_kaggle = X_dataset_Admission_Predict_kaggle.shape[1]

df_report, mean_measures_L1_Admission_Predict_kaggle = L1_Outer_CV(3, INPUT_DIM_Admission_Predict_kaggle, X_dataset_Admission_Predict_kaggle, y_dataset_Admission_Predict_kaggle, df_report, "Admission_Predict_kaggle")
df_report, mean_measures_RLN_Admission_Predict_kaggle = RLN_Outer_CV(3, INPUT_DIM_Admission_Predict_kaggle, X_dataset_Admission_Predict_kaggle, y_dataset_Admission_Predict_kaggle, df_report, "Admission_Predict_kaggle", False)
#With improvment
df_report, mean_measures_RLN_improvment_Admission_Predict_kaggle = RLN_Outer_CV(3, INPUT_DIM_Admission_Predict_kaggle, X_dataset_Admission_Predict_kaggle, y_dataset_Admission_Predict_kaggle, df_report, "Admission_Predict_kaggle", True)

#Update record mean mse for later statistical test
record_statitic_df_Admission_Predict_kaggle = {'Dataset Name': "Admission_Predict_kaggle", 'Mean Squared Error L1': mean_measures_L1_Admission_Predict_kaggle[6], 'Mean Squared Error RLN':mean_measures_RLN_Admission_Predict_kaggle[6], 'Mean Squared Error RLN improved':mean_measures_RLN_improvment_Admission_Predict_kaggle[6]}
df_report_statistical_test = df_report_statistical_test.append(record_statitic_df_Admission_Predict_kaggle, ignore_index = True)

df_report_statistical_test_Admission_Predict_kaggle = pd.DataFrame(columns = ['Dataset Name', 'Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved'])
df_report_statistical_test_Admission_Predict_kaggle = df_report_statistical_test_Admission_Predict_kaggle.append(record_statitic_df_Admission_Predict_kaggle, ignore_index = True)
df_report.to_csv("df_report_Admission_Predict_kaggle.csv", sep=',')
df_report_statistical_test_Admission_Predict_kaggle.to_csv("df_report_statistical_test_Admission_Predict_kaggle.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_Admission_Predict_kaggle.csv
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_Admission_Predict_kaggle.csv

df_report

Downloading...
From: https://drive.google.com/uc?id=1Y8XB-m_7cMf5OOCqG41PA1OK9tQk10Rh&export=download
To: /content/Admission_Predict_kaggle.csv
100%|██████████| 11.4k/11.4k [00:00<00:00, 6.30MB/s]


False
dataset_Admission_Predict_kaggle
*****************************
**********************************************L1 Outer CV number 1**************************************************************
#####################################L1 Inner CV number 1##############################################
start L1 regularization
##################################### END L1 Inner CV number 1##############################################
#####################################L1 Inner CV number 2##############################################
start L1 regularization
##################################### END L1 Inner CV number 2##############################################
#####################################L1 Inner CV number 3##############################################
start L1 regularization
##################################### END L1 Inner CV number 3##############################################
**********************************************END L1 Outer CV number 1*********************

,Dataset Name,Algorithm Name,Cross Validation [1-10],Hyper-Parameters Values,Mean Squared Error,Mean Absolute Error,Median Absolute Error,Max Error,Explained Variance,Training time,Inference time for 1000 instances
0,kc1_numeric,L1,1,0.59182,56.353232,3.577932,0.832973,30.784375,-0.019732,0.884208,1.348413
1,kc1_numeric,L1,2,0.0939923,40.379562,3.512180,0.874388,19.646297,-0.377182,1.231910,1.463101
2,kc1_numeric,L1,3,0.0939923,96.412440,4.428279,0.714828,37.597858,0.095391,0.985722,1.671553
3,kc1_numeric,L1,4,0.0939923,217.459075,5.521369,0.896466,74.462950,0.454506,0.961105,1.665280
4,kc1_numeric,L1,5,0.619111,53.973454,4.017438,0.413447,21.641087,0.243652,0.988991,1.762037
5,kc1_numeric,RLN,1,"[-9.306122448979592, 6.0]",53.821837,3.257851,0.587679,31.110112,0.051428,11.733151,1.935030
6,kc1_numeric,RLN,2,"[-9.63265306122449, 6.0]",37.462463,3.313215,0.300616,19.938206,0.002976,11.784944,2.044965
7,kc1_numeric,RLN,3,"[-10.122448979591837, 6.0]",90.348424,4.229661,0.681615,36.589796,0.143673,12.465107,2.182484
8,kc1_numeric,RLN,4,"[-9.63265306122449, 6.0]",290.376603,6.180665,0.919864,84.994876,0.275196,12.178495,2.195424
9,kc1_numeric,RLN,5,"[-9.63265306122449, 6.0]",68.366722,4.473863,0.488034,23.804154,0.074276,12.318184,2.400892


# Upload reports to GoogleDrive

In [ ]:
df_report_statistical_test.to_csv("df_report_statistical_test_full.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_statistical_test_full.csv

Uploading file: df_report_statistical_test_full.csv


In [ ]:
df_report.to_csv("df_report_all_cv_full.csv", sep=',')
!gupload --to '1mFXx4gAzYZF8dfn0yLL5et3SPgsRSpzH' df_report_all_cv_full.csv

Uploading file: df_report_all_cv_full.csv


#Statistical tests - Friedman test and Post Hoc

In [ ]:
import gdown
output = "dataset_algorithm mse.xlsx"
url = "https://drive.google.com/uc?id=10MaydXu1LFr8wA-rl-uHNpjPD_wRecl6&export=download"
# https://drive.google.com/file/d/10MaydXu1LFr8wA-rl-uHNpjPD_wRecl6/view?usp=sharing
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=10MaydXu1LFr8wA-rl-uHNpjPD_wRecl6&export=download
To: /content/dataset_algorithm mse.xlsx
100%|██████████| 10.2k/10.2k [00:00<00:00, 1.28MB/s]


'dataset_algorithm mse.xlsx'

In [ ]:
import pandas as pd
import numpy as np

dataset_statistical_test = pd.read_excel(output)
dataset_statistical_test

,Dataset Name,Mean Squared Error L1,Mean Squared Error RLN,Mean Squared Error RLN improved
0,vineyard,3.523591e+02,3.454964e+02,3.376260e+02
1,triazines,6.424585e+26,2.635958e+26,1.452768e+27
2,aids,9.222231e-01,7.545389e-01,6.515910e-01
3,analcatdata_negotiation,3.706357e+01,3.538038e+01,3.359224e+01
4,analcatdata_runshoes,9.777535e+00,9.970432e+00,8.253552e+00
5,sensory,1.679671e+02,1.661453e+02,1.674804e+02
6,rmftsa_ladata,2.823438e+04,2.823480e+04,2.824737e+04
7,pwLinear,1.934485e+01,1.943673e+01,1.924121e+01
8,pharynx,4.895224e+05,4.895779e+05,4.890748e+05
9,machine_cpu,3.691066e+04,3.684093e+04,3.678936e+04


In [ ]:
from scipy.stats import friedmanchisquare

friednam_results = friedmanchisquare(dataset_statistical_test['Mean Squared Error L1'], dataset_statistical_test['Mean Squared Error RLN'], dataset_statistical_test['Mean Squared Error RLN improved'])
friednam_results

FriedmanchisquareResult(statistic=6.300000000000011, pvalue=0.04285212686703994)

In [ ]:
df_to_rank = dataset_statistical_test[['Mean Squared Error L1', 'Mean Squared Error RLN', 'Mean Squared Error RLN improved']]
df_to_rank

,Mean Squared Error L1,Mean Squared Error RLN,Mean Squared Error RLN improved
0,3.523591e+02,3.454964e+02,3.376260e+02
1,6.424585e+26,2.635958e+26,1.452768e+27
2,9.222231e-01,7.545389e-01,6.515910e-01
3,3.706357e+01,3.538038e+01,3.359224e+01
4,9.777535e+00,9.970432e+00,8.253552e+00
5,1.679671e+02,1.661453e+02,1.674804e+02
6,2.823438e+04,2.823480e+04,2.824737e+04
7,1.934485e+01,1.943673e+01,1.924121e+01
8,4.895224e+05,4.895779e+05,4.890748e+05
9,3.691066e+04,3.684093e+04,3.678936e+04


In [ ]:
def ranking_model(results):
    ranking = pd.DataFrame(columns=results.columns)
    for i in range(results.shape[0]):
        ranking.loc[i, results.iloc[i].rank(ascending=False).index]=results.iloc[i].rank(ascending=True)
    return ranking

In [ ]:
ranked_df = ranking_model(df_to_rank)
ranked_df

,Mean Squared Error L1,Mean Squared Error RLN,Mean Squared Error RLN improved
0,3,2,1
1,2,1,3
2,3,2,1
3,3,2,1
4,2,3,1
5,3,1,2
6,1,2,3
7,2,3,1
8,2,3,1
9,3,2,1


In [ ]:
average_ranked_L1 = ranked_df['Mean Squared Error L1'].mean()
average_ranked_L1

2.3

In [ ]:
average_ranked_RLN = ranked_df['Mean Squared Error RLN'].mean()
average_ranked_RLN

2.15

In [ ]:
average_ranked_RLN_improved = ranked_df['Mean Squared Error RLN improved'].mean()
average_ranked_RLN_improved

1.55

In [ ]:
import scikit_posthocs as sp
import numpy as np

#combine three groups into one array
data = np.array([dataset_statistical_test['Mean Squared Error L1'], dataset_statistical_test['Mean Squared Error RLN'], dataset_statistical_test['Mean Squared Error RLN improved']])

#perform Nemenyi post-hoc test
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2
0,1.000000,0.872165,0.046567
1,0.872165,1.000000,0.139445
2,0.046567,0.139445,1.000000
